In [ ]:
import random
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark.sql.functions import col,to_date, year, month, unix_timestamp, lit
from functools import reduce

In [ ]:
storage_account = "stworkshopbigdata"

In [ ]:
# Configure Storage Account connection with SAS Token (current Token valid until 22.06.2025)
#! This is Databricks config for running the jobs on one of the existing clusters
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", "sp=racwdl&st=2023-06-21T16:43:59Z&se=2025-06-22T00:43:59Z&spr=https&sv=2022-11-02&sr=c&sig=0oCOUnwh%2B%2BoZzG39oOIVmNHm1HGvuwMz%2F8pyRSlVdOo%3D")

### 1.2 Read Yellow Taxi Data Frames from CSV in storage account

In [ ]:
raw_base_path = f"abfs://raw@{storage_account}.dfs.core.windows.net/"

In [ ]:
def read_taxi_dataframe(spark, base_path, year):
    path = base_path + f"{year}_Yellow_Taxi_Trip_Data.csv"
    df = spark.read.format("csv") \
        .option("inferSchema", "true") \
        .option("header", "true") \
        .option("sep", ",") \
        .load(path)
    return df

In [ ]:
df_2020 = read_taxi_dataframe(spark, raw_base_path, "2020")
df_2021 = read_taxi_dataframe(spark, raw_base_path, "2021")

### 2. Combine data frames

In [ ]:
def merge_dataframes(dataframes):
    ''' Args: pass a list of dataframes'''
    merged_df = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), dataframes)

    return merged_df

In [ ]:
df = merge_dataframes([df_2021, df_2020])

### 3. Clean up and add columns

In [ ]:
df = df.withColumn("pickup_datetime", f.to_timestamp(col("tpep_pickup_datetime"), "MM/dd/yyyy hh:mm:ss a"))\
       .withColumn("dropoff_datetime", f.to_timestamp(col("tpep_dropoff_datetime"), "MM/dd/yyyy hh:mm:ss a"))\
       .withColumn("pickup_date", f.to_date(col("pickup_datetime"))) \
       .withColumn("dropoff_date", f.to_date(col("dropoff_datetime"))) \
       .withColumn('month', f.month(col("pickup_datetime"))) \
       .withColumn('year', f.year(col("pickup_datetime"))) \
       .withColumn('pickup_timestamp', f.unix_timestamp(col("pickup_datetime"))) \
       .withColumn('dropoff_timestamp', f.unix_timestamp(col("dropoff_datetime"))) \
       .drop("tpep_pickup_datetime","tpep_dropoff_datetime")

In [ ]:
# original data frames have some faulty records with dates in the past or in the future, hence a filter:
df = df.filter(df['year'].isin(2020, 2021))

In [ ]:
# define columns order
df_cols = ['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'pickup_date', 'dropoff_date', 'month', 'year', 'pickup_timestamp', 'dropoff_timestamp']

In [ ]:
df_updated = df.select(*df_cols)

### 4. Create list of strings column and include as a string

In [ ]:
streets = ['1st Ave', '1st Ave Loop', '1st Pl', '1st St', '2nd Ave', '2nd Pl', '2nd St', '3rd Ave', '3rd Ave', '3rd Pl', '4th Ave', '5th Ave', '6 Ave', '7th Ave S', '7th Ave', '8th Ave', '9th Ave', '10th Ave', '11th Ave', '12th Ave', '59th St Bridge', '65th Street Transverse', '6th Ave', '65th Street Transverse', '72nd St Transverse (aka Terrace Drive)', '79th Street Transverse', '85th St Transverse', '86th Street Transverse', '97th Street Transverse', '102nd Street Crossing', '145th St Bridge', 'Abingdon Square', 'Abraham Kazan St', 'Abraham Kazan St', 'Abraham Pl', 'Absecon Rd', 'Academy St', 'Adam Clayton Powell Jr Blvd', 'Adrian Ave', 'Alan St', 'Albany St', 'Allen St', 'Amsterdam Ave', 'Amsterdam Ave', 'Andes Rd', 'Ann St', 'Arden St', 'Asser Levy Pl', 'Astor Pl', 'Attorney St', 'Audubon Ave', 'Avenue A', 'Avenue B', 'Avenue C', 'Avenue D', 'Avenue Of The Finest', 'Avenue Of The Strongest', 'Bank St', 'Barclay St', 'Barrow St', 'Barrow St', 'Barry Rd', 'Baruch Dr', 'Baruch Pl', 'Battery Park Underpass', 'Battery Pl', 'Baxter St', 'Bayard St', 'Beach St', 'Beak St', 'Bear Rd', 'Beaver St', 'Bedford St', 'Beekman Pl', 'Beekman St', 'Bennett Ave', 'Benson St', 'Bernard Baruch Way', 'Bethune St', 'Bialystoker Pl', 'Bialystoker St', 'Bleecker St', 'Bloomfield St', 'Bogardus Pl', 'Bond St', 'Bradhurst Ave', 'Bridge St', 'Broad St', 'Broadway', 'Broadway Terrace', 'Brooklyn Battery Tunnel', 'Brooklyn Bridge', 'Broome St', 'Cabrini Blvd', 'Canal St', 'Cannon St', 'Carder Rd', 'Cardinal Hayes Pl', 'Carlisle St', 'Carmine St', 'Cartigan Rd', 'Cathedral Pkwy', 'Catherine Ln', 'Catherine Slip', 'Catherine St', 'Cedar St', 'Center Dr', 'Central Park Driveway', 'Central Park N', 'Central Park S', 'Central Park West', 'Central Rd', 'Centre Market Pl', 'Centre St', 'Centre St', 'Chambers St', 'Charles Ln', 'Charles St', 'Charlton St', 'Chatham Square', 'Cherokee Pl', 'Cherry St', 'Chincoteague Rd', 'Chisum Pl', 'Chittenden Ave', 'Christopher St', 'Chrystie St', 'Church St', 'Claremont Ave', 'Clarkson St', 'Clayton Rd', 'Cleveland Pl', 'Cliff St', 'Clinton St', 'Coenties Alley', 'Coenties Slip', 'Collister St', 'Colonel Robert Magaw Pl', 'Columbia St', 'Columbus Ave', 'Columbus Cir', 'Comfort Rd', 'Commerce St', 'Convent Ave', 'Cooper Square', 'Cooper St', 'Cornelia St', 'Cortlandt Alley', 'Cortlandt St', 'Craig Rd N', 'Craig Rd S', 'Crosby St', 'Ctrl Park Powell Blvd Appr', 'Cumming St', 'De Peyster St', 'Delancey St', 'Delancey St S', 'Desbrosses St', 'Dey St', 'Division Rd', 'Division St', 'Dominick St', 'Dongan Pl', 'Donnellon Square', 'Doris C Freedman Pl', 'Dover St', 'Downing St', 'Doyers St', 'Duane St', 'Duke Ellington Blvd', 'Dutch St', 'Dyckman St', 'Dyer Ave', 'E 100th St', 'E 101st St', 'E 102nd St', 'E 103rd St', 'E 104th St', 'E 105th St', 'E 106th St', 'E 108th St', 'E 109th St', 'E 10th St', 'E 10th St', 'E 110th St', 'E 111th St', 'E 112th St', 'E 113th St', 'E 114th St', 'E 115th St', 'E 116th St', 'E 117th St', 'E 118th St', 'E 119th St', 'E 11th St', 'E 120th St', 'E 121st St', 'E 122nd St', 'E 123rd St', 'E 124th St', 'E 125th St', 'E 126th St', 'E 127th St', 'E 128th St', 'E 129th St', 'E 12th St', 'E 130th St', 'E 131st St', 'E 132nd St', 'E 135th St', 'E 138th St', 'E 13th St', 'E 14th St', 'E 15th St', 'E 16th St', 'E 17th St', 'E 18th St', 'E 19th St', 'E 1st St', 'E 20th St', 'E 21st St', 'E 22nd St', 'E 23rd St', 'E 24th St', 'E 25th St', 'E 26th St', 'E 27th St', 'E 28th St', 'E 29th St', 'E 2nd St', 'E 30th St', 'E 31st St', 'E 32nd St', 'E 33rd St', 'E 34th St', 'E 35th St', 'E 36th St', 'E 37th St', 'E 38th St', 'E 39th St', 'E 3rd St', 'E 40th St', 'E 41st St', 'E 42nd St', 'E 43rd St', 'E 44th St', 'E 45th St', 'E 46th St', 'E 47th St', 'E 48th St', 'E 49th St', 'E 4th St', 'E 4th St', 'E 4th Walk', 'E 50th St', 'E 51st St', 'E 51st Street Pedestrian Crossing', 'E 52nd St', 'E 53rd St', 'E 54th St', 'E 55th St', 'E 56th St', 'E 57th St', 'E 58th St', 'E 59th St', 'E 5th St', 'E 60th St', 'E 61st St', 'E 62nd St', 'E 63rd St', 'E 64th St', 'E 65th St', 'E 66th St', 'E 67th St', 'E 68th St', 'E 69th St', 'E 6th St', 'E 70th St', 'E 71st St', 'E 72nd St', 'E 73rd St', 'E 74th St', 'E 75th St', 'E 76th St', 'E 77th St', 'E 78th St', 'E 79th St', 'E 7th St', 'E 80th St', 'E 81st St', 'E 82nd St', 'E 83rd St', 'E 84th St', 'E 85th St', 'E 86th St', 'E 87th St', 'E 88th St', 'E 89th St', 'E 8th St', 'E 90th St', 'E 91st St', 'E 92nd St', 'E 93rd St', 'E 94th St', 'E 95th St', 'E 96th St', 'E 97th St', 'E 98th St', 'E 99th St', 'E 107th St', 'E 9th St', 'E Broadway', 'E Houston St', 'E Rd', 'E Rd', 'E River Dr', 'E River Piers', 'East Dr', 'East End Ave', 'Edens Alley', 'Edgar Allen Poe St', 'Edgar St', 'Edgecombe Ave', 'Edward M Morgan Pl', 'Eldert Ln', 'Eldridge St', 'Elizabeth St', 'Elk St', 'Ellwood St', 'Enright Rd', 'Ericsson Pl', 'Escanaba Rd', 'Esplanade Gardens Plaza', 'Essex St', 'Evans Rd', 'Exchange Alley', 'Exchange Pl', 'Exchange Plaza', 'Extra Pl', 'F D R Drive Service Rd E', 'Fairview Ave', 'Fashion Ave', 'Fdr Dr', 'Fdr Dr', 'Fdr Dr', 'Fdr Dr', 'Fdr Drive Service Rd E', 'Federal Plaza', 'Ferry Line Rd', 'Fletcher St', 'Forsyth St', 'Fort Charles Pl', 'Fort George Ave', 'Fort George Hill', 'Fort Washington Ave', 'Frankfort St', 'Franklin D. Roosevelt East River Dr', 'Franklin Pl', 'Franklin St', 'Frawley Cir', 'Fred Douglass Cir', 'Frederick Douglass Blvd', 'Frederick Douglass Cir', 'Freedom Pl', 'Freeman Alley', 'Front St', 'Ft Tryon Pl', 'Gansevoort St', 'Gay St', 'George Washington Bridge', 'Gold St', 'Gouverneur Ln', 'Gouverneur Slip E', 'Gouverneur Slip W', 'Gouverneur St', 'Gouvernour St', 'Governors Is', 'Gracie Square', 'Gracie Terrace', 'Gramercy Park E', 'Gramercy Park N', 'Gramercy Park S', 'Gramercy Park W', 'Grand Army Plaza', 'Grand St', 'Great Jones Alley', 'Great Jones St', 'Greene Ave', 'Greene St', 'Greenwich Ave', 'Greenwich Ct', 'Greenwich St', 'Gresham Rd', 'Grove Ct', 'Grove St', 'Grove St', 'Gustave L. Levy Pl', 'Half Moon Rd', 'Hamilton Pl', 'Hamilton Pl', 'Hamilton Terrace', 'Hancock Pl', 'Hanover Square', 'Hanover St', 'Harlem River Dr', 'Harrison St', 'Haven Ave', 'Hay Rd', 'Hayes Pl', 'Henderson Pl', 'Henry Browne Block', 'Henry Hudson Pkwy', 'Henry St', 'Henry St', 'Henshaw St', 'Hester St', 'Hillside Ave', 'Hogan Pl', 'Holland Tunnel', 'Horatio St', 'Howard St', 'Hubert St', 'Hudson St', 'Hwy 9', 'Hwy 9', 'Icarus Rd', 'Indian Rd', 'Interstate 495', 'Interstate 678', 'Interstate 78', 'Irving Pl', 'Isham St', 'Jackson St', 'Jacobus Pl', 'James St', 'Jane St', 'Jay St', 'Jefferson Ave', 'Jefferson St', 'Jersey St', 'Joe Dimaggio Hwy', 'John Jay Byrne Bridge', 'John St', 'Jones Alley', 'Jones St', 'Joseph P Ward St', 'Jumel Pl', 'Jumel Terrace', 'Kenmare St', 'Kimball Rd - Governors Island National Monument', 'Kimmel Rd', 'King Ave', 'King St', 'La Salle St', 'Lafayette Ct', 'Lafayette St', 'Laguardia Pl', 'Laight St', 'Laurel Hill Terrace', 'Legion Memorial Square', 'Lenox Ave', 'Lenox Terrace Pl', 'Leonard St', 'Leroy St', 'Lewis St', 'Lexington Ave', 'Liberty Pl', 'Liberty St', 'Lillian Wald Dr', 'Lincoln Center Plaza', 'Lincoln Tunnel', 'Lispenard St', 'Little W St', 'Little West 12th St', 'Lloyd E Dickens Pl', 'Loisaida Ave', 'Ludlow St', 'Luis Munoz Marin Blvd', 'Mac Dougal Alley', 'Macdougal St', 'Macombs Dam Bridge', 'Macombs Pl', 'Madison Ave', 'Madison Ave', 'Madison Ave Bridge', 'Madison Square N', 'Madison St', 'Magaw Pl', 'Maiden Ln', 'Main St', 'Malcolm X Blvd', 'Mangin St', 'Manhattan', 'Manhattan Ave', 'Marble Hill Ave', 'Marble Hill Ln', 'Margaret Corbin Dr', 'Marginal St', 'Market Slip', 'Market St', 'Marketfield St', 'Martin Luther King Blvd', 'Mc Kenna Square', 'Mc Nally Plaza', 'Mercer St', 'Meyer Square', 'Mill Ln', 'Milligan Pl', 'Minetta Ln', 'Minetta St', 'Mitchell Pl', 'Monroe St', 'Monroe St', 'Montgomery St', 'Moore St', 'Morningside Ave', 'Morningside Dr', 'Morris St', 'Morton St', 'Mosco St', 'Mott St', 'Mt Carmel Pl', 'Mt Morris Park W', 'Mulberry St', 'Murray St', 'N Moore St', 'Nagle Ave', 'Nassau St', 'Nathan D Perlman Pl', 'New York 9a', 'New York Plaza', 'Norfolk St', 'North End Ave', 'Odell Clark Pl', 'Old Broadway', 'Old Slip', 'Oliver St', 'Orchard St', 'Overlook Terrace', 'Owasco Rd', 'Paladino Ave', 'Park Ave', 'Park Ave', 'Park Ave S', 'Park Pl', 'Park Row', 'Park Terrace E', 'Park Terrace W', 'Parks Ave', 'Patchin Pl', 'Payson Ave', 'Pearl St', 'Pell St', 'Penn Plaza', 'Perry St', 'Peter Cooper Rd', 'Pike Slip', 'Pike St', 'Pine St', 'Pinehurst Ave', 'Pitt St', 'Platt St', 'Plaza Lafayette', 'Pleasant Ave', 'Pomander Walk', 'Post Ave', 'Prince St', 'Queens Midtown Tunnel', 'Queensboro Bridge', 'Randalls Is Rd', 'Reade St', 'Rector Pl', 'Rector St', 'Renwick St', 'Ridge St', 'River Rd', 'River Terrace', 'Riverside Blvd', 'Riverside Dr', 'Riverside Dr', 'Riverside Dr W', 'Riverview Terrace', 'Rivington St', 'Robert F Wagner Sr Pl', 'Rockefeller Plaza', 'Roosevelt Island Bridge', 'Rose St', 'Rutgers Slip', 'Rutgers St', 'Rutherford Pl', 'S Pinehurst Ave', 'S Street Viaduct', 'S William St', 'Samuel Dickstein Plaza', 'Seaman Ave', 'Seminary Row', 'Sheridan Square', 'Sheriff St', 'Sherman Ave', 'Short Ave', 'Shubert Alley', 'Sickles St', 'Sniffen Ct', 'South End Ave', 'South St', 'South St', 'Spring St', 'Spruce St', 'St Andrews Plaza', 'St Clair Pl', 'St James Pl', 'St Johns Ln', 'St Lukes Pl', 'St Marks Pl', 'St Nicholas Ave', 'St Nicholas Pl', 'St Nicholas Terrace', 'Staff St', 'Stanton St', 'Staple St', 'State St', 'Stone St', 'Stonewall Pl', 'Stuyvesant Oval', 'Stuyvesant St', 'Stuyvesant Walk', 'Suffolk St', 'Suffolk Street Community Garden', 'Sullivan St', 'Sullivan St', 'Sutton Pl S', 'Sutton Square', 'Sylvan Ct', 'Sylvan Pl', 'Sylvan Terrace', 'Szold Pl', 'Tampa Rd', 'Taras Shevchenko Pl', 'Terrace Dr', '(aka 72nd St Transverse Rd)', 'Terrace View Ave', 'Thayer St', 'The Mall', 'Theatre Alley', 'Thelonius Monk Cir', 'Thomas St', 'Thompson St', 'Tiemann Pl', 'Trans-manhattan Expy', 'Trimble Pl', 'Trinity Pl', 'Tudor City Pl', 'Tunnel Entrance St', 'Tunnel Exit St', 'U.s. Highway 1', 'Union Square E', 'Union Square W', 'United Nations Plaza', 'University Pl', 'Van Corlear Pl', 'Vandam St', 'Vanderbilt Ave', 'Varick St', 'Vermilyea Ave', 'Vesey Pl', 'Vesey St', 'Vestry St', 'W 100th St', 'W 101st St', 'W 102nd St', 'W 103rd St', 'W 104th St', 'W 105th St', 'W 106th St', 'W 107th St', 'W 108th St', 'W 109th St', 'W 10th St', 'W 110th St', 'W 111th St', 'W 112th St', 'W 113th St', 'W 114th St', 'W 115th St', 'W 116th St', 'W 117th St', 'W 118th St', 'W 119th St', 'W 11th St', 'W 120th St', 'W 121st St', 'W 122nd St', 'W 123rd St', 'W 124th St', 'W 125th St', 'W 126th St', 'W 127th St', 'W 128th St', 'W 129th St', 'W 12th St', 'W 130th St', 'W 131st St', 'W 132nd St', 'W 133rd St', 'W 134th St', 'W 135th St', 'W 136th St', 'W 137th St', 'W 138th St', 'W 139th St', 'W 13th St', 'W 13th St', 'W 140th St', 'W 141st St', 'W 142nd St', 'W 143rd St', 'W 144th St', 'W 145th St', 'W 146th St', 'W 147th St', 'W 148th St', 'W 149th St', 'W 14th St', 'W 150th St', 'W 151st St', 'W 152nd St', 'W 153rd St', 'W 154th St', 'W 155th St', 'W 156th St', 'W 157th St', 'W 158th St', 'W 159th St', 'W 15th St', 'W 160th St', 'W 161st St', 'W 162nd St', 'W 163rd St', 'W 164th St', 'W 165th St', 'W 165th St', 'W 166th St', 'W 167th St', 'W 168th St', 'W 169th St', 'W 16th St', 'W 170th St', 'W 171st St', 'W 172nd St', 'W 173rd St', 'W 174th St', 'W 175th St', 'W 176th St', 'W 177th St', 'W 178th St', 'W 179th St', 'W 17th St', 'W 180th St', 'W 181st St', 'W 182nd St', 'W 183rd St', 'W 184th St', 'W 185th St', 'W 186th St', 'W 187th St', 'W 188th St', 'W 189th St', 'W 18th St', 'W 190th St', 'W 191st St', 'W 192nd St', 'W 193rd St', 'W 196th St', 'W 19th St', 'W 201st St', 'W 202nd St', 'W 203rd St', 'W 204th St', 'W 205th St', 'W 206th St', 'W 207th St', 'W 208th St', 'W 20th St', 'W 211th St', 'W 212th St', 'W 213th St', 'W 214th St', 'W 215th St', 'W 216th St', 'W 217th St', 'W 218th St', 'W 219th St', 'W 21st St', 'W 220th St', 'W 225th St', 'W 227th St', 'W 228th St', 'W 22nd St', 'W 23rd St', 'W 24th St', 'W 25th St', 'W 26th St', 'W 27th St', 'W 28th St', 'W 29th St', 'W 30th St', 'W 31st St', 'W 32nd St', 'W 33rd St', 'W 34th St', 'W 35th St', 'W 36th St', 'W 37th St', 'W 38th St', 'W 39th St', 'W 3rd St', 'W 40th St', 'W 41st St', 'W 42nd St', 'W 43rd St', 'W 44th St', 'W 45th St', 'W 46th St', 'W 47th St', 'W 48th St', 'W 49th St', 'W 4th St', 'W 4th St', 'W 50th St', 'W 51st St', 'W 52nd St', 'W 53rd St', 'W 54th St', 'W 55th St', 'W 56th St', 'W 57th St', 'W 58th St', 'W 59th St', 'W 60th St', 'W 61st Dr', 'W 61st St', 'W 62nd St', 'W 63rd St', 'W 64th St', 'W 65th St', 'W 66th St', 'W 67th St', 'W 68th St', 'W 69th St', 'W 70th St', 'W 71st St', 'W 72nd St', 'W 73rd St', 'W 74th St', 'W 75th St', 'W 76th St', 'W 77th St', 'W 78th St', 'W 79st Transverse Rd', 'W 79th St', 'W 80th St', 'W 81st St', 'W 82nd St', 'W 83rd St', 'W 84th St', 'W 85th St', 'W 86th St', 'W 87th St', 'W 88th St', 'W 89th St', 'W 8th St', 'W 90th St', 'W 91st St', 'W 92nd St', 'W 93rd St', 'W 94th St', 'W 95th St', 'W 96th St', 'W 97th St', 'W 98th St', 'W 99th St', 'W 9th St', 'W Broadway', 'W Houston St', 'W Rd', 'W Thames St', 'W Washington Pl', 'Wadsworth Ave', 'Wadsworth Terrace', 'Walker St', 'Wall St', 'Wards Is Rd', 'Warren St', 'Warren St', 'Warren St', 'Warren St', 'Washington Bridge', 'Washington Mews', 'Washington Pl', 'Washington Square E', 'Washington Square N', 'Washington Square S', 'Washington Square W', 'Washington St', 'Washington St', 'Washington St', 'Washington Terrace', 'Water St', 'Watts St', 'Waverly Pl', 'Weehawken St', 'West End Ave', 'West St', 'West St', 'West St', 'Wheeler Ave', 'White St', 'Whitehall St', 'Willett St', 'William St', 'Williamsburg Bridge', 'Willis Ave Bridge', 'Wooster St', 'Worth St', 'Wyckoff Ave', 'Yeaton Rd', 'York Ave', 'Administration Hill', 'Asylum Rd', 'Crowninshield St', 'Depeyster St -the College Of Mount Saint Vincent', 'Emmet St', 'Erben Ave', 'Hanus St', 'Mcgowan St', 'Patterson St', 'Reeder St', 'Shepard Ave', 'Wadhams St', '10th Ave S', '11th Ave', '12th Ave', '13th Ave', '1st Ave', '1st St', '1st St', '226th Dr S', '229th Dr N', '229th St Dr S', '2nd Ave', '2nd St', '3rd Ave', '3rd St', '4th Ave', '5th Ave', '6th Ave', '7th Ave', '9th Ave', 'A St', 'Abbot St', 'Acorn Pl', 'Adams Pl', 'Adams St', 'Adee Ave', 'Adee Dr', 'Adler Pl', 'Admiral Ct', 'Admiral Ln', 'Agar Pl', 'Agawam N', 'Alan Pl', 'Albany Crescent', 'Alcott Pl', 'Alden Park', 'Alden Pl', 'Alderbrook Rd', 'Aldrich St', 'Aldus St', 'Alexander Ave', 'Alexander Ave', 'Allen Pl', 'Allerton Ave', 'Amberson Ave', 'Amethyst St', 'Ampere Ave', 'Amundson Ave', 'Amundson Ave', 'Anderson Ave', 'Andrews Ave N', 'Andrews Ave S', 'Anthony Ave', 'Anthony J Griffin Pl', 'Antin Pl', 'Aqueduct Ave', 'Aqueduct Ave', 'Aqueduct Ave E', 'Aqueduct Ave W', 'Archer Rd', 'Archer St', 'Arlington Ave', 'Armand Pl', 'Arnow Ave', 'Arnow Pl', 'Arthur Ave', 'Asch Loop', 'Aster Pl', 'Astor Ave', 'Austin Pl', 'Avenue A', 'Avenue B', 'Avenue C', 'Avenue D', 'Avenue E', 'Avenue F', 'Avenue St John', 'B Ave', 'B St', 'Bailey Ave', 'Bailey Pl', 'Bainbridge Ave', 'Baisley Ave', 'Baker Ave', 'Balcom Ave', 'Baldwin St', 'Baldwin St', 'Balsam Pl', 'Banes Ct', 'Bantam Pl', 'Banyer Pl', 'Barker Ave', 'Barkley Ave', 'Barnes Ave', 'Barnett Pl', 'Barrett Ave', 'Barretto St', 'Barry St', 'Bartholdi St', 'Bartow Ave', 'Bassett Ave', 'Bassford Ave', 'Bathgate Ave', 'Bay Shore Ave', 'Bay St', 'Baychester Ave', 'Bayview Ave', 'Beach Ave', 'Beach St', 'Beach St', 'Beacon Cir', 'Beacon Ln', 'Beaumont Ave', 'Beck St', 'Bedford Park Blvd', 'Bedford Park Blvd W', 'Beech Pl', 'Beech Terrace', 'Beech Tree Ln', 'Beekman Ave', 'Belden St', 'Bell Ave', 'Bell Ave', 'Bellamy Loop', 'Belmont Ave', 'Belmont St', 'Benchley Pl', 'Benedict Ave', 'Benson St', 'Bergen Ave', 'Berkeley Ave', 'Berkwit Terrace', 'Betts Ave', 'Bevy Pl', 'Billingsley Terrace', 'Bingham Rd', 'Birchall Ave', 'Bissel Ave', 'Bivona St', 'Blackrock Ave', 'Blackstone Ave', 'Blackstone Pl', 'Blair Ave', 'Blondell Ave', 'Bogart Ave', 'Boller Ave', 'Bolton Ave', 'Bolton St', 'Bonner Pl', 'Boone Ave', 'Boscobel Pl', 'Boston Post Rd', 'Boston Rd', 'Boston Rd', 'Botanical Square', 'Botanical Square N', 'Botanical Square S', 'Bouck Ave', 'Bowne St', 'Boyd Ave', 'Boynton Ave', 'Bradford Ave', 'Bradley St', 'Bradley St', 'Bradley Terrace', 'Brady Ave', 'Brandt Pl', 'Bridge St', 'Briggs Ave', 'Briggs Ave', 'Brinsmade Ave', 'Bristow St', 'Britton St', 'Broadway', 'Bronx Blvd', 'Bronx Park E', 'Bronx Park Rd', 'Bronx Park S', 'Bronx River Ave', 'Bronx River Parkway', 'Bronx River Pkwy', 'Bronx River Pkwy', 'Bronx St', 'Bronxdale Ave', 'Bronxwood Ave', 'Brook Ave', 'Broun Pl', 'Brown Ln', 'Brown Pl', 'Browns Ln', 'Bruce Ave', 'Bruckner Blvd', 'Bruckner Expy', 'Bruner Ave', 'Brush Ave', 'Bryant Ave', 'Buchanan Pl', 'Buck St', 'Buckley St', 'Buhre Ave', 'Bullard Ave', 'Burke Ave', 'Burnet Pl', 'Burr Ave', 'Bush St', 'Bushnell Pl', 'Bussing Ave', 'Bussing Pl', 'Butler Pl', 'Butterick Ave', 'Buttrick Ave', 'Byron Ave', 'C Ave', 'C St', 'Caesar Pl', 'Calhoun Ave', 'Cambreleng Ave', 'Cambridge Ave', 'Cameron Pl', 'Camp St', 'Campbell Dr', 'Canal Pl', 'Canal St W', 'Cannon Pl', 'Capuchin Way', 'Cardinal Spellman Pl', 'Carlisle Pl', 'Carpenter Ave', 'Carroll Pl', 'Carroll St', 'Carter Ave', 'Carver Loop', 'Caryl Ave', 'Casals Pl', 'Casanova St', 'Casler Pl', 'Castle Hill Ave', 'Catalpa Pl', 'Cauldwell Ave', 'Cayuga Ave', 'Cedar Ave', 'Cedar Ln', 'Cedar Pl', 'Central Park Ave', 'Centre St', 'Chaffee Ave', 'Charlotte St', 'Chatterton Ave', 'Chesbrough Ave', 'Chester St', 'Chestnut St', 'Chisholm St', 'Choctaw Pl', 'Cicero Ave', 'Cincinnatus Ave', 'City Island Ave', 'City Island Rd', 'Claflin Ave', 'Clarence Ave', 'Clarke Pl W', 'Clason Point Ln', 'Clay Ave', 'Clay Ave', 'Clementine St', 'Clifford Pl', 'Clifford Pl E', 'Clifton Ave', 'Clinton Ave', 'Clinton Pl', 'Close Ave', 'Co-op City Blvd', 'Coddington Ave', 'Colden Ave', 'Coles Ln', 'Colgate Ave', 'College Ave', 'College Rd', 'Collis Pl', 'Colonial Ave', 'Commerce Ave', 'Commonwealth Ave', 'Compton Ave', 'Concord Ave', 'Concourse Village E', 'Concourse Village W', 'Connell Pl', 'Conner St', 'Continental Ave', 'Coogans Pl', 'Cooper Ave', 'Cooper Pl', 'Corlear Ave', 'Cornell Ave', 'Cornell Pl', 'Corsa Ave', 'Cortlandt St', 'Coster St', 'Cottage Pl', 'Country Club Rd', 'Courter Ave', 'Courtlandt Ave', 'Cox Ave', 'Cranford Ave', 'Cranford Ave', 'Craven St', 'Crawford Ave', 'Crescent Ave', 'Creston Ave', 'Crimmins Ave', 'Croes Ave', 'Croes Pl', 'Cromwell Ave', 'Crosby Ave', 'Cross Bronx Expressway Service Rd', 'Cross Bronx Expy', 'Cross Bronx Service Rd N', 'Cross Bx Service Road Exd', 'Cross St', 'Crotona Ave', 'Crotona Park E', 'Crotona Park N', 'Crotona Park S', 'Crotona Pkwy', 'Crotona Pl', 'Cruger Ave', 'Curtiss Ave', 'Cypress Ave', 'Cyrus Pl', 'D Ave', 'D St', 'Daisy Pl', 'Daly Ave', 'Daniel St', 'Dare Pl', 'Dark St', 'Darrow Pl', 'Dash Pl', 'David Sheridan Plaza', 'Davidson Ave', 'Davis Ave', 'Davit Ct', 'Dawson St', 'De Kruif Pl', 'De Reimer Ave', 'De Witt Pl', 'Dean Ave', 'Debs Pl', 'Decatur Ave', 'Deepwater Way', 'Defoe Pl', 'Dekalb Ave', 'Delafield Ave', 'Delafield Pl', 'Delancey Pl', 'Delano Ave', 'Delanoy Ave', 'Delavall Ave', 'Demeyer St', 'Depot Pl', 'Depot Pl Appr', 'Devanney Square', 'Devoe Ave', 'Devoe Terrace', 'Dewey Ave', 'Dewitt Pl', 'Deyo St', 'Dickinson Ave', 'Dickinson Ave', 'Digney Ave', 'Dill Pl', 'Ditmars St', 'Dock St', 'Dock St', 'Dodgewood Rd', 'Dogwood Dr', 'Donizetti Pl', 'Donofrio Square', 'Doris St', 'Dorothea Pl', 'Dorsey St', 'Douglas Ave', 'Dr Martin Luther King Jr Blvd', 'Dr Theodore Kazimiroff Blvd', 'Drainage St', 'Drake Park S', 'Drake St', 'Dreiser Loop', 'Dudley Ave', 'Duncan St', 'Duncomb Ave', 'Dune Ct', 'Dupont St', 'Duryea Ave', 'Duryea Ave', 'Dwight Pl', 'Dyre Ave', 'E 132nd St', 'E 133rd St', 'E 134th St', 'E 135th St', 'E 136th St', 'E 137th St', 'E 139th St', 'E 140th St', 'E 141st St', 'E 142nd St', 'E 143rd St', 'E 144th St', 'E 145th St', 'E 146th St', 'E 147th St', 'E 148th St', 'E 149th St', 'E 150th St', 'E 151st St', 'E 152nd St', 'E 153rd St', 'E 154th St', 'E 155th St', 'E 156th St', 'E 157th St', 'E 158th St', 'E 159th St', 'E 160th St', 'E 161st St', 'E 162nd St', 'E 163rd St', 'E 164th St', 'E 165th St', 'E 166th St', 'E 167th St', 'E 168th St', 'E 169th St', 'E 170th St', 'E 171st St', 'E 172nd St', 'E 173rd St', 'E 174th St', 'E 175th St', 'E 176th St', 'E 177th St', 'E 178th St', 'E 179th St', 'E 180th St', 'E 181st St', 'E 182nd St', 'E 183rd St', 'E 184th St', 'E 185th St', 'E 186th St', 'E 187th St', 'E 188th St', 'E 189th St', 'E 190th St', 'E 191st St', 'E 192nd St', 'E 193rd St', 'E 194th St', 'E 195th St', 'E 196th St', 'E 197th St', 'E 198th St', 'E 199th St', 'E 201st St', 'E 202nd St', 'E 203rd St', 'E 204th St', 'E 205th St', 'E 206th St', 'E 207th St', 'E 208th St', 'E 209th St', 'E 210th St', 'E 211th St', 'E 212th St', 'E 213th St', 'E 214th St', 'E 215th St', 'E 216th St', 'E 217th St', 'E 218th St', 'E 219th St', 'E 220th St', 'E 221st St', 'E 222nd St', 'E 223rd St', 'E 224th St', 'E 225th St', 'E 226th St', 'E 227th St', 'E 228th St', 'E 229th Dr N', 'E 229th Dr S', 'E 229th St', 'E 230th St', 'E 231st St', 'E 232nd St', 'E 233rd St', 'E 234th St', 'E 235th St', 'E 236th St', 'E 237th St', 'E 238th St', 'E 239th St', 'E 240th St', 'E 241st St', 'E 241st St', 'E 242nd St', 'E 242nd St', 'E 243rd St', 'E 243rd St', 'E Bay Ave', 'E Burnside Ave', 'E Clarke Pl', 'E Clifford Pl', 'E Fordham Rd', 'E Gun Hill Rd', 'E Kingsbridge Rd', 'E Kingsbridge Rd', 'E Mosholu Pkwy N', 'E Mosholu Pkwy S', 'E Tremont Ave', 'Eagle Ave', 'Eames Pl', 'Earhart Ln', 'Earley St', 'East Ave', 'Eastburn Ave', 'Eastchester Pl', 'Eastchester Rd', 'Echo Pl', 'Eden Terrace', 'Edenwald Ave', 'Edenwald Ave', 'Edge St', 'Edgehill Ave', 'Edgemere St', 'Edgemere St', 'Edgewater Rd', 'Edison Ave', 'Edison Ave', 'Edsall Ave', 'Edson Ave', 'Edward L Grant Hwy', 'Edwards Ave', 'Effingham Ave', 'Eger Pl', 'Einstein Loop', 'Einstein Loop E', 'Einstein Loop N', 'Einstein Loop S', 'Elder Ave', 'Elgar Pl', 'Elinor Pl', 'Elliot Pl', 'Ellis Ave', 'Ellis Pl', 'Ellison Ave', 'Ellsworth Ave', 'Ellsworth Ave', 'Elm Dr', 'Elm Pl', 'Elm Tree Ln', 'Elm Tree Ln', 'Elsmere Pl', 'Elton Ave', 'Ely Ave', 'Emerson Ave', 'Erdman Pl', 'Ericson Pl', 'Erskine Ave', 'Erskine Pl', 'Esplanade', 'Esplanade Ave', 'Euclid Ave', 'Evelyn Pl', 'Evergreen Ave', 'Exterior St', 'Faile St', 'Fairfax Ave', 'Fairfield Ave', 'Fairmont Pl', 'Fairmount Ave', 'Fairmount Pl', 'Fanshaw Ave', 'Faraday Ave', 'Farragut St', 'Father Zeiser Pl', 'Fearn Pl', 'Featherbed Ln', 'Fenton Ave', 'Fenway N', 'Fenway S', 'Fern Pl', 'Ferris Pl', 'Field Pl', 'Fielding St', 'Fieldston Rd', 'Fieldston Terrace', 'Fillmore St', 'Findlay Ave', 'Fink Ave', 'Fish Ave', 'Fleet Ct', 'Fletcher Pl', 'Flint Ave', 'Folin St', 'Food Center Dr', 'Food Center Rd', 'Food Dr', 'Foote Ave', 'Ford St', 'Fordham Pl', 'Fordham St', 'Forest Ave', 'Forest Ave', 'Forest Rd', 'Forster Pl', 'Fort Independence St', 'Fowler Ave', 'Fox St', 'Fox Terrace', 'Franklin Ave', 'Freeman St', 'Frisby Ave', 'Fteley Ave', 'Fuller St', 'Fulton Ave', 'Furman Ave', 'G St', 'Gabriel Dr', 'Gale Pl', 'Garden Pl', 'Garden St', 'Garfield St', 'Garrett Pl', 'Garrison Ave', 'Gates Pl', 'George Pl', 'George St', 'Geranium Pl', 'Gerard Ave', 'Gerber Pl', 'Gertland Pl', 'Giegerich Pl', 'Gifford Ave', 'Gilbert Pl', 'Gildersleeve Ave', 'Giles Pl', 'Gillespie Ave', 'Givan Ave', 'Gleason Ave', 'Glebe Ave', 'Glennon Pl', 'Glover Ave', 'Glover St', 'Goble Pl', 'Godwin Terrace', 'Good Pl', 'Goodridge Ave', 'Goulden Ave', 'Gouverneur Ave', 'Gouverneur Pl', 'Grace Ave', 'Graff Ave', 'Grand Ave', 'Grand Concourse', 'Grand Concourse', 'Grandview Pl', 'Grant Ave', 'Grant Ave', 'Greene Pl', 'Grenada Pl', 'Greystone Ave', 'Grinnell Pl', 'Griswold Ave', 'Grosvenor Ave', 'Grote St', 'Guerlain St', 'Guion Pl', 'Gunther Ave', 'Hadley Ave', 'Haight Ave', 'Hall Of Fame Terrace', 'Hall Pl', 'Halleck St', 'Halperin Ave', 'Halsey St', 'Hamilton Pl', 'Hammersley Ave', 'Hampden Pl', 'Hancock Ave', 'Harding Ave', 'Harding Ave', 'Harding Camp', 'Harding Park', 'Harding Park Camp', 'Harlem River Park Bridge', 'Harper Ave', 'Harper Ave', 'Harper Ct', 'Harrington Ave', 'Harrison Ave', 'Harrod Ave', 'Harrod Pl', 'Hart St', 'Haskin St', 'Hatting Pl', 'Havemeyer Ave', 'Haviland Ave', 'Hawkins St', 'Hawkstone St', 'Hawthorne Ave', 'Hawthorne Dr', 'Hawthorne St', 'Hazel Pl', 'Hazen St', 'Heath Ave', 'Heathcote Ave', 'Hegney Pl', 'Hennessy Pl', 'Henry Hudson Pkwy', 'Henry Hudson Pkwy E', 'Henry Hudson Pkwy W', 'Henwood Pl', 'Hering Ave', 'Herkimer Pl', 'Hermany Ave', 'Herschell St', 'Hewitt Pl', 'Hicks St', 'Hill Ave', 'Hillman Ave', 'Hobart Ave', 'Hoe Ave', 'Hoffman St', 'Holland Ave', 'Hollers Ave', 'Holly Pl', 'Hollywood Ave', 'Holt Pl', 'Home St', 'Homer Ave', 'Hone Ave', 'Honeywell Ave', 'Hornaday Pl', 'Horton St', 'Hosmer Ave', 'Houghton Ave', 'Howe Ave', 'Hoxie St', 'Hubbell St', 'Hudson Manor Terrace', 'Hugh J Grant Cir', 'Hughes Ave', 'Huguenot Ave', 'Hull Ave', 'Hunt Ave', 'Hunter Ave', 'Hunter Ave', 'Huntington Ave', 'Hunts Point Ave', 'Hunts Point County Op Market', 'Husson Ave', 'Hutchinson Ave', 'Hutchinson River Pkwy', 'Hutchinson River Pkwy E', 'Huxley Ave', 'Hwy 9', 'Hyatt Ave', 'I St', 'I-95', 'Independence Ave', 'Independence Ave', 'Indian Rd', 'Indian Trail', 'Industrial St', 'Interstate 278', 'Interstate 295', 'Interstate 87', 'Interstate 895', 'Intervale Ave', 'Inwood Ave', 'Irvine St', 'Irving Ave', 'Irving Pl', 'Irwin Ave', 'Iselin Ave', 'Ittner Pl', 'Ivy Pl', 'J St', 'Jackson Ave', 'Jackson Ave', 'Jarrett Pl', 'Jarvis Ave', 'Jasamine Pl', 'Jay Pl', 'Jefferson Pl', 'Jennings St', 'Jerome Ave', 'Jesup Ave', 'Jesup Pl', 'Johnson Ave', 'Johnson St', 'K St', 'Kappock St', 'Katonah Ave', 'Kearney Ave', 'Kelly St', 'Kennellworth Pl', 'Kepler Ave', 'Kilroe St', 'Kimball Ave', 'Kimberly Pl', 'King Ave', 'King Ave', 'King St', "Kings' College Place", 'Kingsbridge Ave', 'Kingsbridge Terrace', 'Kingsland Ave', 'Kingsland Plaza', 'Kinnear Pl', 'Kinsella St', 'Kipp Rd', 'Kirby St', 'Kirk St', 'Knapp St', 'Knolls Creek', 'Knolls Crescent', 'Knox Pl', 'Kossuth Ave', 'L St', 'La Fontaine Ave', 'Lacombe Ave', 'Laconia Ave', 'Ladd Rd', 'Lafayette Ave', 'Lafontaine Ave', 'Lakeview Pl', 'Lakewood Pl', 'Lamport Pl', 'Landing Rd', 'Landscape Ave', 'Lane Ave', 'Lasalle Ave', 'Latting St', 'Laurel Dr', 'Laurie Ave', 'Lawton Ave', 'Layton Ave', 'Lebanon St', 'Lee Ave', 'Lee St', 'Leeward Ln', 'Leggett Ave', 'Lehman Terrace', 'Leighton Ave', 'Leland Ave', 'Leroy Ave', 'Lester St', 'Libby Pl', 'Library Ave', 'Liebig Ave', 'Light St', 'Lighting Pl', 'Lincoln Ave', 'Linden Ave', 'Linden Dr', 'Lisbon Pl', 'Livingston Ave', 'Locust Ave', 'Locust Point Dr', 'Lodovick Ave', 'Logan Ave', 'Lohengrin Pl', 'Longfellow Ave', 'Longmeadow Rd', 'Longstreet Ave', 'Longwood Ave', 'Loomis St', 'Loop Rd', 'Lorillard Pl', 'Loring Pl N', 'Loring Pl S', 'Louis 9 Blvd', 'Lowell St', 'Lowerre Pl', 'Lucerne St', 'Lurting Ave', 'Lustre St', 'Lydig Ave', 'Lyman Pl', 'Lyon Ave', 'Lyvere St', 'Mac Donough Pl', 'Mac Questen Pkwy N', 'Mac Questen Pkwy S', 'Maccracken Ave', 'Mace Ave', 'Maclay Ave', 'Macombs Dam Bridge', 'Macombs Rd', 'Macy Pl', 'Magenta St', 'Magnolia Pl', 'Mahan Ave', 'Main St', 'Maitland Ave', 'Major Deegan Expy', 'Major Deegan Serv Rd', 'Manhattan College Pkwy', 'Manida St', 'Manning St', 'Manor Ave', 'Mansion St', 'Mapes Ave', 'Maple Ave', 'Maple Dr', 'Maran Pl', 'Marcy Pl', 'Marina Dr', 'Marine St', 'Marion Ave', 'Market St', 'Marmion Ave', 'Marolla Pl', 'Martha Ave', 'Martha Ave', 'Marvin Pl', 'Matilda Ave', 'Matthews Ave', 'Mayflower Ave', 'Mc Donald St', 'Mc Owen Ave', 'Mcclellan St', 'Mcgraw Ave', 'Mclean Ave', 'Mead St', 'Meagher Ave', 'Melrose Ave', 'Melville St', 'Mercy College Pl', 'Merriam Ave', 'Merrill St', 'Merritt Ave', 'Merry Ave', 'Metcalf Ave', 'Metropolitan Av - Red Oak Dr', 'Metropolitan Ave', 'Metropolitan Oval', 'Meyers St', 'Mickle Ave', 'Middletown Rd', 'Mildred Pl', 'Miles Ave', 'Milton Pl', 'Minerva Pl', 'Minford Pl', 'Minnieford Ave', 'Miriam St', 'Mitchell Pl', 'Mohegan Ave', 'Monroe Ave', 'Monroe St', 'Monsignor Halpin Pl', 'Monterey Ave', 'Montgomery Ave', 'Montgomery Pl', 'Monticello Ave', 'Morgan Ave', 'Morris Ave', 'Morris Park Ave', 'Morrison Ave', 'Morton Pl', 'Mosholu Ave', 'Mosholu Pkwy', 'Mosholu Pkwy N', 'Mosholu Pkwy S', 'Msgr Cahill Pl', 'Mt Eden Ave', 'Mt Eden Pkwy', 'Mt Hope Pl', 'Mt St Ursula Pl', 'Mulford Ave', 'Muliner Ave', 'Mullan Pl', 'Mullen Pl', 'Mulvey Ave', 'Mundy Ln', 'Mundy Ln', 'Murdock Ave', 'Murray Ct', 'N Chestnut Dr', 'N Oak Dr', 'Napier Ave', 'Naples Terrace', 'Narragansett Ave', 'Needham Ave', 'Neill Ave', 'Nelson Ave', 'Neptune Ct', 'Neptune Ln', 'Nereid Ave', 'Nereid Ave', 'Netherland Ave', 'New England Thruway', 'New York State Thruway', 'Newbold Ave', 'Newman Ave', 'Newport Ave', 'Niell Ave', 'Noble Ave', 'Noell Ave', 'North St', 'Norton Ave', 'Nuvern Ave', 'Nuvern Ave', 'Ny Orphan Aym Rd', 'Oak Ave', 'Oak Ln', 'Oak Point Ave', 'Oak Terrace', 'Oak Tree Pl', 'Oakland Pl', 'Oakley St', 'Obrien Ave', 'Odell St', 'Ogden Ave', 'Ogden Ave', 'Ohm Ave', 'Old Albany Post Rd', 'Old Jerome Ave', 'Old Kingsbridge Rd', 'Old White Plains Rd', 'Olinville Ave', 'Oliver Pl', 'Olmstead Ave', 'Oneida Ave', 'Oneill Pl', 'Orchard Beach Rd', 'Orloff Ave', 'Osbourne Pl', 'Osgood St', 'Osman Pl', 'Otis Ave', 'Outlook Ave', 'Overing St', 'Oxford Ave', 'P St', 'Paine St', 'Palisade Ave', 'Palisade Pl', 'Palmer Ave', 'Park Ave', 'Park Dr', 'Park Dr', 'Park Ln', 'Park Ln', 'Parkchester Rd', 'Parker St', 'Parkside Pl', 'Parkview Ave', 'Parkview Terrace', 'Parkway N', 'Parsifal Pl', 'Patterson Ave', 'Paul Ave', 'Paulding Ave', 'Paulis Pl', 'Pawnee Pl', 'Peace St', 'Pearl St', 'Pearsall Ave', 'Peartree Ave', 'Pelham Bay Park W', 'Pelham Bridge Rd', 'Pelham Pkwy - Eastchester Rd', 'Pelham Pkwy N', 'Pelham Pkwy S', 'Pelham Prkw. S ../../view.php?place=Pelham Prkw. S #1a, Bronx, ny, USA1a', 'Pell Pl', 'Penfield St', 'Pennyfield Ave', 'Pennyfield Camp', 'Perot St', 'Perry Ave', 'Peters Pl', 'Phelan Pl', 'Philip Ave', 'Pierce Ave', 'Pilgrim Ave', 'Pilot St', 'Pinchot Pl', 'Pine Dr', 'Pinkney Ave', 'Pitman Ave', 'Plaza Pl', 'Plimpton Ave', 'Ploughmans Bush', 'Plymouth Ave', 'Poe Pl', 'Polo Pl', 'Pond Pl', 'Pontiac Pl', 'Ponton Ave', 'Popham Ave', 'Poplar Ave', 'Poplar St', 'Post Rd', 'Powell Ave', 'Powers Ave', 'Pratt Ave', 'Prentiss Ave', 'Priory Ln', 'Prospect Ave', 'Prospect Pl', 'Provost Ave', 'Pugsley Ave', 'Purdy St', 'Puritan Ave', 'Putnam Ave W', 'Putnam Pl', 'Q St', 'Quarry Rd', 'Quimby Ave', 'Quincy Ave', 'R St', 'Radcliff Ave', 'Radio Dr', 'Rae St', 'Railroad Terrace', 'Randall Ave', 'Randolph Pl', 'Rawlins Ave', 'Redwood Oak Dr', 'Reed Pl', 'Reeds Mill Ln', 'Regina Pl', 'Reiss Pl', 'Research Ave', 'Reservoir Ave', 'Reservoir Oval E', 'Reservoir Oval W', 'Reservoir Pl', 'Rev James A Polite Ave', 'Revere Ave', 'Reverend Kenneth L Folkes Way', 'Review Pl', 'Reville St', 'Reynold St', 'Reynolds Ave', 'Rhinelander Ave', 'Richardson Ave', 'Richman Plaza', 'Rider Ave', 'Ridge Pl', 'Ridge Pl', 'Rikers Is Br Appr', 'Rikers Is Branch', 'Ritter Pl', 'River Ave', 'River Crest Rd', 'River Rd', 'Rivercrest Rd', 'Riverdale Ave', 'Riverdale Ave', 'Riverdale Ave', 'Riverdale Yacht Club', 'Roberts Ave', 'Robertson Pl', 'Robertson St', 'Robertson St', 'Robinson Ave', 'Rochambeau Ave', 'Rochelle St', 'Rockledge Ave', 'Rockwood St', 'Rodman Pl', 'Roebling Ave', 'Rogers Pl', 'Rohr Pl', 'Rombouts Ave', 'Roosevelt Ave', 'Roosevelt Ave', 'Roosevelt Pl', 'Ropes Ave', 'Rose Feiss Blvd', 'Rosedale Ave', 'Roselle St', 'Rosewood St', 'Rowe St', 'Rowland St', 'Rudd Pl', 'Ryawa Ave', 'Ryer Ave', 'S Devoe Ave', 'S Macquesten Pkwy', 'S Oak Dr', 'S Southern Blvd', 'Sackett Ave', 'Sagamore St', 'Sampson Ave', 'Sands Pl', 'Saratoga Ave', 'Saxon Ave', 'Scenic Pl', 'Schieffelin Ave', 'Schieffelin Pl', 'Schley Ave', 'Schofield St', 'Schorr Pl', 'Schurz Ave', 'Schuyler Pl', 'Schuyler Terrace', 'Scott Pl', 'Screvin Ave', 'Seabury Ave', 'Seabury Pl', 'Secor Ave', 'Secor Ave', 'Seddon St', 'Sedgewick Ave', 'Sedgwick Ave', 'Selwyn Ave', 'Seminole Ave', 'Seminole St', 'Seneca Ave', 'Senger Pl', 'Seton Ave', 'Seton Ave', 'Seward Ave', 'Sexton Pl', 'Seymour Ave', 'Shakespeare Ave', 'Sheridan Ave', 'Sheridan Expy', 'Sheriff S Byrd Pl', 'Sherman Ave', 'Sherman Ave', 'Shore', 'Shore Dr', 'Shore Rd', 'Shoreview Cir', 'Shrady Pl', 'Siegfried Pl', 'Sigma Pl', 'Silver Beach Pl', 'Silver St', 'Simpson St', 'Sommer Pl', 'Soundview Ave', 'Soundview Terrace', 'South St', 'Southern Blvd', 'Spaulding Ln', 'Spencer Ave', 'Spencer Dr', 'Spencer Pl', 'Spencer Terrace', 'Split Rock Rd', 'Splitrock Rd', 'Spofford Ave', 'Spring Rd', "St Ann's Pl", 'St Anns Ave', 'St Barnabas Pl', 'St Georges Crescent', "St Joseph's Way", 'St Lawrence Ave', 'St Marks Pl', 'St Marys St', 'St Ouen St', 'St Pauls Ave', 'St Pauls Pl', 'St Pauls Pl', 'St Peters Ave', 'St Raymonds Ave', 'St Theresa Ave', 'Stadium Ave', 'Stanton Ct', 'Starboard Ct', 'Starling Ave', 'State Highway 22', 'Stearns St', 'Stebbins Ave', 'Stedman Pl', 'Steenwick Ave', 'Stell Pl', 'Step St', 'Stephens Ave', 'Sterling Ave', 'Steuben Ave', 'Steuben Ave', 'Stevens Pl', 'Stevenson Pl', 'Stickball Blvd', 'Stickney Pl', 'Stillwell Ave', 'Story Ave', 'Strang Ave', 'Stratford Ave', 'Strong St', 'Suburban Pl', 'Sullivan Pl', 'Summit Ave', 'Summit Pl', 'Sunset Blvd', 'Sunset Trail', 'Surf Dr', 'Sutherland St', 'Swinton Ave', 'Sycamore Ave', 'Sycamore Dr', 'Sylvan Ave', 'T St', 'Tan Pl', 'Taylor Ave', 'Teller Ave', 'Tenbroeck Ave', 'Tenney Pl', 'Terrace Pl', 'Terrace Pl', 'Terrace St', 'Terrace St', 'The Mall At Bay Plaza', 'Thieriot Ave', 'Thorns Ln', 'Throggs Neck Blvd', 'Throggs Neck Expy', 'Throgmorton Ave', 'Throgs Neck Bridge', 'Throop Ave', 'Thurman Munson Way', 'Thwaites Pl', 'Tibbett Ave', 'Tibbetts Rd', 'Tiebout Ave', 'Tiemann Ave', 'Tier St', 'Tierney Pl', 'Tiffany St', 'Tilden St', 'Tillotson Ave', 'Timpson Pl', 'Tinton Ave', 'Tomlinson Ave', 'Topping Ave', 'Torry Ave', 'Townsend Ave', 'Trafalgar Pl', 'Tratman Ave', 'Trinity Ave', 'Truxton St', 'Tryon Ave', 'Tudor Pl', 'Tulfan Terrace', 'Turnbull Ave', 'Turneur Ave', 'Tyndall Ave', 'Tyndall Ave', 'U St', 'U.s. 1', 'Undercliff Ave', 'Underhill Ave', 'Union Ave', 'Unionport Rd', 'University Ave', 'Us Army Reservation', 'Valentine Ave', 'Valhalla Dr', 'Valles Ave', 'Van Buren St', 'Van Cortlandt Ave E', 'Van Cortlandt Ave W', 'Van Cortlandt Park Ave', 'Van Cortlandt Park E', 'Van Cortlandt Park S', 'Van Hoesen Ave', 'Van Nest Ave', 'Vance St', 'Varian Ave', 'Victor St', 'Viele Ave', 'Villa Ave', 'Vincent Ave', 'Vireo Ave', 'Virgil Pl', 'Virginia Ave', 'Vista Pl', 'Vreeland Ave', 'Vyse Ave', 'W 161st St', 'W 162nd St', 'W 163rd St', 'W 164th St', 'W 165th St', 'W 166th St', 'W 167th St', 'W 168th St', 'W 169th St', 'W 170th St', 'W 171st St', 'W 172nd St', 'W 174th St', 'W 175th St', 'W 176th St', 'W 177th St', 'W 179th St', 'W 180th St', 'W 181st St', 'W 182nd St', 'W 183rd St', 'W 184th St', 'W 188th St', 'W 190th St', 'W 192nd St', 'W 193rd St', 'W 195th St', 'W 197th St', 'W 205th St', 'W 229th St', 'W 230th St', 'W 231st St', 'W 232nd St', 'W 233rd St', 'W 234th St', 'W 235th St', 'W 236th St', 'W 237th St', 'W 238th St', 'W 239th St', 'W 240th St', 'W 242nd St', 'W 244th St', 'W 245th St', 'W 246th St', 'W 247th St', 'W 249th St', 'W 250th St', 'W 251st St', 'W 252nd St', 'W 253rd St', 'W 254th St', 'W 255th St', 'W 256th St', 'W 258th St', 'W 259th St', 'W 260th St', 'W 261st St', 'W 262nd St', 'W 263rd St', 'W 263rd St', 'W Burnside Ave', 'W Delano Ave', 'W Farms Rd', 'W Farms Square Plaza', 'W Fordham Rd', 'W Gun Hill Rd', 'W Kingsbridge Rd', 'W Kingsbridge Rd', 'W Mosholu Pkwy N', 'W Mosholu Pkwy S', 'W Tremont Ave', 'Wade Square', 'Wakefield Ave', 'Waldo Ave', 'Wales Ave', 'Wallace Ave', 'Walnut Ave', 'Walton Ave', 'Ward Ave', 'Waring Ave', 'Washington Ave', 'Washington Ave', 'Washington Blvd', 'Waterbury Ave', 'Waterloo Pl', 'Waters Ave', 'Waters Pl', 'Watson Ave', 'Watt Ave', 'Wavehill Driveway', 'Wayne Ave', 'Wchester Ave', 'Webb Ave', 'Webster Ave', 'Weeks Ave', 'Weiher Ct', 'Wellesley Ave', 'Wellman Ave', 'Wenner Pl', 'West Ave', 'West St', 'West St', 'Westchester Ave', 'Westchester Ave', 'Westchester Ave', 'Westchester Ave', 'Westchester Ave', 'Westchester Square', 'Westervelt Ave', 'Whalen Ave', 'Wheeler Ave', 'White Plains Rd', 'White Plains Rd', 'Whitehall Pl', 'Whitlock Ave', 'Whittemore Ave', 'Whittier St', 'Wickham Ave', 'Wilcox Ave', 'Wilder Ave', 'Wildwood Ave', 'Wilkins Ave', 'Wilkinson Ave', 'Willett Ave', 'William Ave', 'William Pl', 'Williamsbridge Rd', 'Willis Ave', 'Willow Ave', 'Willow Ln', 'Wilson Ave', 'Windsor Pl', 'Windward Ln', 'Winters St', 'Wissman Ave', 'Wood Ave', 'Wood Rd', 'Woodhull Ave', 'Woodlawn Ave', 'Woodmansten Pl', 'Woodycrest Ave', 'Worthen St', 'Wright Ave', 'Wyatt St', 'Wythe Pl', 'X St', 'Yankee Mall', 'Yates Ave', 'Young Ave', 'Yznaga Pl', 'Zerega Ave', 'Zulette Ave', 'Hillcrest Rd', 'Nassau Pl', 'Richmond Ct', 'Suffolk Ave', '1 St', '10 St', '10th Ave', '10th St', '12th Ave', '13rd Ave', '1st Ave', '1st St', '2 Ct', '2 St', '2nd Ave', '2nd St', '3 Ave', '3 St', '3rd Ct', '3rd St', '4 Corners Rd', '4 Ct', '4 St', '4th Ave', '4th St', '5th', '5th St', '6th', '6th Ave', '7 Gables Rd', '7 St', '7th St', '8 St', '8th Ave', '8th St', '9 St', '9th St', 'A St', 'Aaron Ln', 'Abbey Rd', 'Abbott St', 'Abby Pl', 'Abingdon Ave', 'Abingdon Ct', 'Acacia Ave', 'Academy Ave', 'Academy Pl', 'Ackerman St', 'Acorn Ct', 'Acorn St', 'Ada Dr', 'Ada Pl', 'Adam Ct', 'Adams Ave', 'Addison Ave', 'Adelaide Ave', 'Adele Ct', 'Adele St', 'Adelphi Ave', 'Adlai Cir', 'Adlers Ln', 'Adlers Way', 'Admiralty Loop', 'Adrianne Ln', 'Adrienne Ln', 'Adrienne Pl', 'Agnes Pl', 'Agrig Ct', 'Ainsworth Ave', 'Akron Plaza', 'Akron St', 'Alabama Pl', 'Alan Loop', 'Alaska Pl', 'Alaska St', 'Alban St', 'Albee Ave', 'Albert Ct', 'Albert St', 'Alberta Ave', 'Albion Pl', 'Albourne Ave', 'Albourne Ave E', 'Albright St', 'Alcott St', 'Alden Pl', 'Alderwood Pl', 'Alex Cir', 'Alexander Ave', 'Alexander Way', 'Alexandra Pl', 'Algonkin St', 'Allegro St', 'Allen Ct', 'Allen Pl', 'Allendale Rd', 'Allentown Ln', 'Allison Ave', 'Allison Pl', 'Almond St', 'Alpine Ave', 'Alpine Ct', 'Altamont St', 'Altavista Ct', 'Altavista Pl', 'Alter Ave', 'Altoona Ave', 'Alverson Ave', 'Alverson Loop', 'Alvine Ave', 'Alysia Ct', 'Amador St', 'Amanda Ct', 'Amaron Ln', 'Amazon Ct', 'Ambassador Ln', 'Amber St', 'Amboy Rd', 'Amelia Ct', 'Amherst Ave', 'Amity Pl', 'Amsterdam Ave', 'Amsterdam Pl', 'Amy Ct', 'Amy Ln', 'Anaconda St', 'Anchor Pl', 'Anchorage Ln', 'Anderson Ave', 'Anderson Pl', 'Anderson St', 'Andes Pl', 'Andrea Ct', 'Andrea Pl', 'Andrease St', 'Andrews Ave', 'Andrews St', 'Andros Ave', 'Androvette Ave', 'Androvette St', 'Angelina Ct', 'Anita St', 'Anjali Loop', 'Ann St', 'Annadale Rd', 'Annfield Ct', 'Anthony J Crecca Jr Plaza', 'Anthony St', 'Appleby Ave', 'Arbor Ct', 'Arbutus Ave', 'Arbutus Way', 'Arc Pl', 'Arcadia Pl', 'Archwood Ave', 'Arden Ave', 'Ardmore Ave', 'Ardsley St', 'Area Pl', 'Argonne St', 'Arielle Ln', 'Arkansas Ave', 'Arlene Ct', 'Arlene St', 'Arlington Ave', 'Arlington Ct', 'Arlington Pl', 'Arlo Rd', 'Armour Pl', 'Armstrong Ave', 'Arnold St', 'Arnprior St', 'Arrowood Ct', 'Arthur Ave', 'Arthur Ct', 'Arthur Kill Rd', 'Artillary Rd', 'Artillery Rd', 'Ascot Ave', 'Ash Pl', 'Ashland Ave', 'Ashland Ave E', 'Ashley Ln', 'Ashton Dr', 'Ashwood Ct', 'Ashworth Ave', 'Aspen Knolls Way', 'Aspinwall St', 'Astor Ave', 'Athena Ct', 'Athena Pl', 'Atlantic Ave', 'Atmore Pl', 'Auburn Ave', 'Augusta Ave', 'Aultman Ave', 'Ausable Ave', 'Austin Ave', 'Austin Pl', 'Avalon Ct', 'Ave B', 'Averill Pl', 'Aviston St', 'Aviva Ct', 'Avon Green', 'Avon Ln', 'Avon Pl', 'Aye Ct', 'Aymar Ave', 'Ayres Rd', 'Azalea Ct', 'B St', 'Bache Ave', 'Bache St', 'Baden Pl', 'Bailey Pl', 'Baker Pl', 'Balfour St', 'Ballard Ave', 'Balsam Pl', 'Baltic Ave', 'Baltic St', 'Baltimore St', 'Bamberger Ln', 'Bancroft Ave', 'Bang Terrace', 'Bangor St', 'Bank Pl', 'Bank St', 'Bar Ct', 'Barb St', 'Barbara St', 'Barclay Ave', 'Barclay Cir', 'Bard Ave', 'Baring Pl', 'Barker St', 'Barlow Ct', 'Barnard Ave', 'Baron Blvd', 'Barrett Ave', 'Barrow Pl', 'Barry Ct', 'Barry St', 'Bartlett Ave', 'Barton Ave', 'Bartow Ave', 'Bartow St', 'Bascom Pl', 'Bass St', 'Bates Ave', 'Bath Ave', 'Bathgate St', 'Battery Rd', 'Bay St', 'Bay Street Landing', 'Bayard St', 'Bayonne Bridge', 'Bayside Ln', 'Bayview Ave', 'Bayview Ln', 'Bayview Pl', 'Bayview Terrace', 'Beach Ave', 'Beach Ln', 'Beach Rd', 'Beach St', 'Beachview Ave', 'Beacon Ave', 'Beacon Pl', 'Bear St', 'Beard St', 'Bedell Ave', 'Bedell St', 'Bedford Ave', 'Bee Ct', 'Beebe St', 'Beechwood Ave', 'Beechwood Pl', 'Beekman Cir', 'Beekman St', 'Beethoven St', 'Behan Ct', 'Belair Ln', 'Belair Rd', 'Belair Ridge', 'Belden St', 'Belfast Ave', 'Belfield Ave', 'Belknap St', 'Bell St', 'Bellavista Ct', 'Belle Dr', 'Bellhaven Pl', 'Belmar Dr E', 'Belmar Dr W', 'Belmont Pl', 'Belwood Loop', 'Bement Ave', 'Bement Cmns', 'Bement Ct', 'Benedict Ave', 'Benedict Rd', 'Benjamin Dr', 'Benjamin Pl', 'Bennett Ave', 'Bennett Pl', 'Bennett St', 'Bennington St', 'Benson St', 'Bent St', 'Bentley Ln', 'Bentley St', 'Benton Ave', 'Benton Ct', 'Benziger Ave', 'Beresford Ave', 'Berglund Ave', 'Berkley St', 'Berry Ave', 'Berry Ave W', 'Berry Ct', 'Bertha Pl', 'Bertram Ave', 'Berwick Pl', 'Berwin Ln', 'Beth Pl', 'Bethel Ave', 'Betty Ct', 'Beverly Ave', 'Beverly Rd', 'Bianca Ct', 'Bidwell Ave', 'Billings St', 'Billiou St', 'Billop Ave', 'Bionia Ave', 'Birch Ave', 'Birch Ln', 'Birch Rd', 'Birchard Ave', 'Bishop St', 'Bismark Ave', 'Blackford Ave', 'Blackhorse Ave', 'Blaine Ct', 'Blaise Ct', 'Bland Pl', 'Blanders Ln', 'Bleeker Pl', 'Block St', 'Bloomfield Ave', 'Bloomfield Rd', 'Bloomingdale Rd', 'Blossom Ln', 'Blue Heron Ct', 'Blue Heron Dr', 'Blueberry Ln', 'Blythe Pl', 'Boardwalk Ave', 'Boating Pl', 'Bocce Ct', 'Bodine St', 'Bogert Ave', 'Bogota St', 'Bolivar St', 'Bombay St', 'Bond St', 'Boone St', 'Booth Ave', 'Borman Ave', 'Borough Pl', 'Boscombe Ave', 'Bosworth St', 'Boulder St', 'Boundary Ave', 'Bouton Ln', 'Bovanizer St', 'Bowden St', 'Bowdoin St', 'Bowen St', 'Bower Ct', 'Bowles Ave', 'Bowling Green Pl', 'Boyce Ave', 'Boyd St', 'Boylan St', 'Boyle Pl', 'Boyle St', 'Boynton St', 'Br Ct', 'Brabant St', 'Brad Ln', 'Bradford Ave', 'Bradley Ave', 'Bradley Ct', 'Brady Pl', 'Braisted Ave', 'Brandis Ave', 'Brandis Ln', 'Breezy Ct', 'Brehaut Ave', 'Brenton Pl', 'Brentwood Ave', 'Brewster St', 'Briarcliff Rd', 'Briarwood Rd', 'Bridge Ct', 'Bridgetown St', 'Bridgeview Cir', 'Brielle Ave', 'Brienna Ct', 'Brighton Ave', 'Brighton St', 'Bristol Ave', 'Britton Ave', 'Britton Ln', 'Britton St', 'Broad St', 'Broadway', 'Broken Shell Rd', 'Brook Ave', 'Brook St', 'Brookfield Ave', 'Brooks Pl', 'Brooks Pond Pl', 'Brookside Ave', 'Brookside Loop', 'Brower Ct', 'Brown Ave', 'Brown Pl', 'Brownell St', 'Browning Ave', 'Bruckner Ave', 'Bruno Ln', 'Brunswick St', 'Bryan St', 'Bryant Ave', 'Bryson Ave', 'Buchanan Ave', 'Buel Ave', 'Buffalo St', 'Buffington Ave', 'Bunnell Ct', 'Burbank Ave', 'Burchard Ct', 'Burden Ave', 'Burgher Ave', 'Burgher Ave N', 'Burke Ave', 'Burnside Ave', 'Burton Ave', 'Burton Ct', 'Bush Ave', 'Butler Ave', 'Butler Blvd', 'Butler Pl', 'Butler St', 'Butler Terrace', 'Butterworth Ave', 'Buttonwood Rd', 'Byran St', 'Byrne Ave', 'C St', 'Cable Way', 'Cabot Pl', 'Caldera Pl', 'Callahan Ln', 'Callan Ave', 'Calvin St', 'Cambria St', 'Cambridge Ave', 'Camden Ave', 'Camden Ct', 'Camden St', 'Cameron Ave', 'Campbell Ave', 'Campus Rd', 'Campus Rd', 'Canal St', 'Candon Ave', 'Candon Ct', 'Cannon Ave', 'Cannon Blvd', 'Cannon Dr', 'Canon Dr', 'Canterbury Ave', 'Canterbury Ct', 'Canton Ave', 'Capellan St', 'Cardiff St', 'Cardinal Ln', 'Carlin St', 'Carlton Ave', 'Carlton Blvd', 'Carlton Ct', 'Carlton Pl', 'Carly Ct', 'Carly Pl', 'Carlyle Green', 'Carlyle St', 'Carmel Ave', 'Carmel Ct', 'Carnegie Ave', 'Caro St', 'Carol Ct', 'Carol Pl', 'Carolina Ct', 'Carolina Pl', 'Carolina Rd', 'Caroline St', 'Carolyn Ct', 'Carpenter Ave', 'Carreau Ave', 'Carroll Pl', 'Carteret St', 'Cary Ave', 'Cascade St', 'Case Ave', 'Cassidy Pl', 'Castleton Ave', 'Castleton Ct', 'Castor Pl', 'Caswell Ave', 'Caswell Ln', 'Catherine Ct', 'Catherine Pl', 'Catherine St', 'Catlin Ave', 'Cattaraugus St', 'Cattaraugus St', 'Cattaraugus St', 'Cayuga Ave', 'Cebra Ave', 'Cecil Ct', 'Cedar Ave', 'Cedar Grove Ave', 'Cedar Grove Ave Exd', 'Cedar Grove Beach Pl', 'Cedar Grove Ct', 'Cedar St', 'Cedar Terrace', 'Cedarcliff Rd', 'Cedarview Ave', 'Cedarwood Ct', 'Celebration Ln', 'Celina Ln', 'Celtic Pl', 'Center Pl', 'Center St', 'Central Ave', 'Centre Ave', 'Challenger Dr', 'Champlain Ave', 'Chance Ave', 'Chandler Ave', 'Chapin Ave', 'Chappell St', 'Charles Ave', 'Charles Ct', 'Charles Pl', 'Charleston Ave', 'Chart Loop', 'Charter Oak Rd', 'Chatham St', 'Chelsea Rd', 'Chelsea St', 'Chemical Ln', 'Cherokee St', 'Cherry Pl', 'Cherrywood Ct', 'Cheryl Ave', 'Chesebrough St', 'Cheshire Pl', 'Chess Loop', 'Chester Ave', 'Chester Pl', 'Chesterfield Ln', 'Chesterton Ave', 'Chestnut Ave', 'Chestnut Cir', 'Chestnut Pl', 'Chestnut St', 'Cheves Ave', 'Chicago Alley', 'Chicago Ave', 'Chisholm St', 'Chrissy Ct', 'Christine Ct', 'Christopher Ln', 'Christopher St', 'Church Ave', 'Church Ln', 'Church St', 'Churchill Ave', 'Cindra Ave', 'Circle Loop', 'Circle Rd', 'City Blvd', 'Claire Ct', 'Claradon Ln', 'Clarence Pl', 'Clarion Ct', 'Clark Ave', 'Clark Ln', 'Clark Pl', 'Clark St', 'Clarke Ave', 'Claudia Ct', 'Clawson St', 'Clay Pit Rd', 'Clayboard St', 'Clayton St', 'Clearmont Ave', 'Clermont Ave', 'Clermont Ct', 'Clermont Pl', 'Cletus St', 'Cleveland Alley', 'Cleveland Ave', 'Cleveland Pl', 'Cleveland St', 'Cliff Ct', 'Cliff St', 'Cliffside Ave', 'Cliffwood Ave', 'Clifton Ave', 'Clifton Rd', 'Clifton St', 'Clinton Ave', 'Clinton B Fiske Ave', 'Clinton Ct', 'Clinton Pl', 'Clinton Rd', 'Clinton St', 'Cloister Pl', 'Clove Lake Pl', 'Clove Lakes Exwy S Service Rd', 'Clove Rd', 'Clove Way', 'Cloverdale Ave', 'Clovis Rd', 'Clyde Pl', 'Coale Ave', 'Coast Guard Dr', 'Coastal Dr', 'Cobblers Ln', 'Coco Ct', 'Coddington Ave', 'Cody Pl', 'Coke St', 'Coldspring Ct', 'Coldsprings Ct', 'Cole St', 'Colfax Ave', 'Colgate Pl', 'Colita Ct', 'College Ave', 'College Ct', 'College Pl', 'Collfield Ave', 'Collyer Ave', 'Colon Ave', 'Colon St', 'Colonial Ave', 'Colonial Ct', 'Colony Ave', 'Colorado St', 'Colton St', 'Columbia Ave', 'Columbus Ave', 'Columbus Pl', 'Comanche Ave', 'Combs Ave', 'Comely St', 'Commerce St', 'Commodore Dr', 'Community Ln', 'Comstock Ave', 'Concord Ln', 'Concord Pl', 'Confederation Pl', 'Conference Ct', 'Conger St', 'Congress St', 'Connecticut St', 'Conrad Ave', 'Constant Ave', 'Continental Pl', 'Convent Ave', 'Conyingham Ave', 'Cooke St', 'Coonley Ct', 'Cooper Ave', 'Cooper Pl', 'Cooper Terrace', 'Copley St', 'Copperflagg Ln', 'Copperleaf Terrace', 'Coral Ct', 'Corbin Ave', 'Cordelia Ave', 'Cornelia Ave', 'Cornelia St', 'Cornell Ave', 'Cornell Pl', 'Cornell St', 'Cornish St', 'Cornwall Ave', 'Corona Ave', 'Correll Ave', 'Corson Ave', 'Cortelyou Ave', 'Cortelyou Pl', 'Cortlandt St', 'Coryn Ct', 'Cosmen St', 'Cottage Ave', 'Cottage Pl', 'Cotter Ave', 'Cottontail Ct', 'Cottonwood Ct', 'Coughlan Ave', 'Country Dr E', 'Country Dr N', 'Country Dr S', 'Country Dr W', 'Country Ln', 'Country Woods Ln', 'County House Rd', 'Coursen Ct', 'Coursen Pl', 'Court Pl', 'Court St', 'Courtney Loop', 'Coventry Loop', 'Coventry Rd', 'Coverly Ave', 'Coverly St', 'Covington Cir', 'Cowen Pl', 'Crabbs Ln', 'Crabtree Ave', 'Crabtree Ln', 'Crafton Ave', 'Craig Ave', 'Cranberry Ct', 'Crane St', 'Cranford Ave', 'Cranford Ct', 'Cranford St', 'Cres Beach Rd', 'Crescent Ave', 'Crescent Beach', 'Crest Loop', 'Creston Pl', 'Creston St', 'Crist St', 'Crittenden Pl', 'Croak Ave', 'Croft Ct', 'Croft Pl', 'Cromer St', 'Cromwell Ave', 'Cromwell Cir', 'Cross St', 'Crossfield Ave', 'Crosshill St', 'Croton Ave', 'Croton Pl', 'Crowell Ave', 'Crown Ave', 'Crown Ct', 'Crown Pl', 'Crystal Ave', 'Crystal Ln', 'Cuba Ave', 'Cubberly Pl', 'Culotta Ln', 'Cunard Ave', 'Cunard Pl', 'Currie Ave', 'Curtis Ave', 'Curtis Ct', 'Curtis Pl', 'Cypress Ave', 'Cypress Crest Ln', 'Cypress Loop', 'D St', 'Daffodil Ct', 'Daffodil Ln', 'Dahlia St', 'Dakota Pl', 'Dale Ave', 'Daleham St', 'Dalemere Rd', 'Dallas St', 'Dalton Ave', 'Dan St', 'Dana St', 'Daniel Low Terrace', 'Daniela Ct', 'Daniella Ct', 'Danny Ct', 'Darcey Ave', 'Darian St', 'Darlington Ave', 'Darnell Ln', 'David Pl', 'David St', 'Davidson Ct', 'Davidson St', 'Davis Ave', 'Davis Ct', 'Dawn Ct', 'Dawson Cir', 'Dawson Ct', 'Dawson Pl', 'Dawson St', 'Dayna Dr', 'De Hart Ave', 'De Ruyter Pl', 'De Soto St', 'De Soto St Exd', 'De Soto St Exd', 'Deal Ct', 'Debbie St', 'Deborah Loop', 'Decatur Ave', 'Decker Ave', 'Deems Ave', 'Deere Park Pl', 'Degroot Pl', 'Deirdre Ct', 'Deisius St', 'Dekalb St', 'Dekay St', 'Del Ray Ct', 'Delafield Ave', 'Delafield Pl', 'Delaware Ave', 'Delaware Ave', 'Delaware Ave', 'Delaware Ave', 'Delaware Pl', 'Delaware St', 'Delford St', 'Delia Ct', 'Dell Ct', 'Dellwood Rd', 'Delmar Ave', 'Delmore St', 'Delphine Terrace', 'Delwit Ave', 'Demopolis Ave', 'Demorest Ave', 'Denise Ct', 'Denker Pl', 'Dennis Torricelli Sr St', 'Denoble Ln', 'Dent Rd', 'Dental Ave', 'Denton Pl', 'Depew Pl', 'Deppe Pl', 'Derby Ct', 'Deserre Ave', 'Destiny Ct', 'Detroit Ave', 'Devens St', 'Devon Loop', 'Devon Pl', 'Dewey Ave', 'Dewey Pl', 'Dewhurst St', 'Dexter Ave', 'Di Marco Pl', 'Dianas Trail', 'Diaz Pl', 'Diaz St', 'Dickie Ave', 'Dierauf St', 'Dina Ct', 'Dinsmore St', 'Direnzo Ct', 'Discala Ln', 'Disosway Pl', 'Ditson St', 'Divine St', 'Dix Pl', 'Dixon Ave', 'Doane Ave', 'Dobbs Pl', 'Dock Rd', 'Dock St', 'Doe Ct', 'Doe Pl', 'Dogwood Dr', 'Dogwood Ln', 'Dole St', 'Dolson Pl', 'Domain St', 'Don Ct', 'Donald Pl', 'Dongan Ave', 'Dongan Hills Ave', 'Dongan St', 'Donley Ave', 'Donna Ct', 'Dora St', 'Dore Ct', 'Doreen Dr', 'Dorit Ct', 'Dorothea Pl', 'Dorothy St', 'Dorval Ave', 'Dorval Pl', 'Doty Ave', 'Douglas Ave', 'Douglas Ct', 'Douglas Rd', 'Dover Green', 'Downes Ave', 'Downey Pl', 'Dr Martin Luther King Jr Expy', 'Drake Ave', 'Draper Pl', 'Dresden Pl', 'Drew Ct', 'Dreyer Ave', 'Driggs St', 'Driprock St', 'Drum Ave', 'Drum Rd', 'Drumgoole Rd E', 'Drumgoole Rd W', 'Drury Ave', 'Drury Ave Exd', 'Dryden Ct', 'Drysdale St', 'Duane Ct', 'Dublin Pl', 'Dubois Ave', 'Dudley Ave', 'Duer Ave', 'Duer Ln', 'Dugdale St', 'Duke Pl', 'Dulancey Ct', 'Dumont Ave', 'Dunbar St', 'Duncan Rd', 'Duncan St', 'Dunham St', 'Dunhill Ln', 'Durant Ave', 'Dustan St', 'Dutchess Ave', 'Dutchess Ave', 'Dutchess Ave', 'Dwarf St', 'Dyson St', 'E Augusta Ave', 'E Brandis Ave', 'E Broadway', 'E Buchanan St', 'E Cheshire Pl', 'E Drumgoole Rd', 'E Figurea Ave', 'E Loop Rd', 'E Macon Ave', 'E Pepkiomen Ave', 'E Perkiomen Ave', 'E Raleigh Ave', 'E Reading Ave', 'E Scranton Ave', 'E Service Rd', 'E Stroud Ave', 'E Tenafly Avenue East', 'E Veterans Rd', 'Eadie Pl', 'Eagan Ave', 'Eagle Rd', 'Earley Ave', 'Earley Pl', 'Eastentry Rd', 'Eastman St', 'Eastwood Ave', 'Eaton Pl', 'Ebbitts Ave', 'Ebbitts St', 'Ebey Ln', 'Ebony St', 'Echo Pl', 'Eddy St', 'Eden Ct', 'Edgar Pl', 'Edgar Terrace', 'Edgegrove Ave', 'Edgewater St', 'Edgewood Rd', 'Edinboro Rd', 'Edison St', 'Edith Ave', 'Edstone Dr', 'Edward Ct', 'Edward Curry Ave', 'Edwin St', 'Egbert Ave', 'Egbert Pl', 'Egmont Pl', 'Egrit Ct', 'El Camino Loop', 'Elaine Ct', 'Elbe Ave', 'Elder Ave', 'Eldridge Ave', 'Eleanor Ln', 'Eleanor Pl', 'Eleanor St', 'Elias Pl', 'Elie Ct', 'Elise Ct', 'Elizabeth Ave', 'Elizabeth Ct', 'Elizabeth Pl', 'Elizabeth Rd', 'Elizabeth St', 'Elk Ct', 'Elkhart St', 'Elks Pl', 'Ella Pl', 'Ellicott Pl', 'Ellington St', 'Ellis Rd', 'Ellis St', 'Ellsworth Ave', 'Ellsworth Pl', 'Elm Pl', 'Elm St', 'Elmbank St', 'Elmhurst Ave', 'Elmira Ave', 'Elmira St', 'Elmtree Ave', 'Elmtree Ave Exd', 'Elmwood Ave', 'Elmwood Park Dr', 'Elson Ct', 'Elson St', 'Eltinge St', 'Eltingville Blvd', 'Elverton Ave', 'Elvin St', 'Elwood Ave', 'Elwood Pl', 'Ely Ave', 'Ely Ct', 'Ely St', 'Emerald Ct', 'Emeric Ct', 'Emerson Ave', 'Emerson Ct', 'Emerson Dr', 'Emily Ct', 'Emily Ln', 'Emmet Ave', 'End Pl', 'Endor Ave', 'Endview St', 'Enfield Pl', 'Engert St', 'Englewood Ave', 'Enright St', 'Erastina Pl', 'Eric Ln', 'Erie St', 'Erika Loop', 'Errington Pl', 'Erwin Ct', 'Escanaba Ave', 'Esmac Ct', 'Essex Dr', 'Essex Dr', 'Estelle Pl', 'Esther Depew St', 'Eton Pl', 'Eugene Pl', 'Eugene St', 'Eunice Pl', 'Eva Ave', 'Evan Pl', 'Evans St', 'Evelyn Pl', 'Everett Ave', 'Everett Pl', 'Evergreen Ave', 'Evergreen St', 'Everton Ave', 'Everton Pl', 'Ewing St', 'Excelsior Ave', 'Exec Way', 'Executive Way', 'Exeter St', 'Eylandt St', 'Faber St', 'Fabian St', 'Fahy Ave', 'Fairbanks Ave', 'Fairfield St', 'Fairlawn Ave', 'Fairlawn Loop', 'Fairmount Ave', 'Fairview Ave', 'Fairview Pl', 'Fairway Ave', 'Fairway Ln', 'Falcon Ave', 'Fancher Pl', 'Fanning St', 'Farraday St', 'Farragut Ave', 'Farrell Ct', 'Father Capodanno Blvd', 'Fawn Ln', 'Fayann Ln', 'Fayette Ave', 'Federal Pl', 'Feldmeyers Ln', 'Felt Ave', 'Felton St', 'Fenway Cir', 'Ferguson Ct', 'Fern Ave', 'Ferndale Ave', 'Ferndale Ct', 'Ferry St', 'Ferry St Exd', 'Ficarelle Dr', 'Fiedler Ave', 'Field St', 'Fields Ave', 'Fieldstone Rd', 'Fieldway Ave', 'Figurea Ave', 'Filbert Ave', 'Filer St', 'Filipe Ln', 'Fillat St', 'Fillmore Ave', 'Fillmore Pl', 'Fillmore St', 'Fine Blvd', 'Fingal St', 'Fingerboard Rd', 'Finlay Ave', 'Finlay St', 'Finley Ave', 'Finley Rd', 'Firth Rd', 'Fisher Ave', 'Fiske Ave', 'Fitzgerald Ave', 'Flagg Ct', 'Flagg Pl', 'Flagship Cir', 'Fleetwood Ave', 'Fletcher St', 'Flint St', 'Florence Pl', 'Florence St', 'Florida Ave', 'Florida Terrace', 'Flower Ave', 'Flower Ct', 'Floyd St', 'Foch Ave', 'Fonda Pl', 'Foote Ave', 'Foothill Ct', 'Ford Pl', 'Forest Ave', 'Forest Ct', 'Forest Green', 'Forest Hill Rd', 'Forest Ln', 'Forest Rd', 'Forest St', 'Fornes Pl', 'Forrestal Ave', 'Forrestal Ct', 'Fort Hill Cir', 'Fort Hill Park', 'Fort Hill Pl', 'Fort Pl', 'Foss Ave', 'Foster Ave', 'Foster Rd', 'Fox Beach Ave', 'Fox Hill Terrace', 'Fox Hunt Ct', 'Fox Ln', 'Foxbeach Ave', 'Foxholm St', 'Fr Capodanno Blvd', 'Francesca Ln', 'Francine Ct', 'Francine Ln', 'Francis Pl', 'Frank Ct', 'Franklin Ave', 'Franklin D Roosevelt Boardwalk', 'Franklin Ln', 'Franklin Pl', 'Fraser St', 'Frean St', 'Frederick St', 'Freeborn St', 'Freedom Ave', 'Freeman Pl', 'Fremont Ave', 'Fremont St', 'Front St', 'Ft Hill Park', 'Fuller Ct', 'Fulton St', 'Furman St', 'Furness Pl', 'Futurity Pl', 'Gabriele Ct', 'Gadsen Pl', 'Gail Ct', 'Gales Ln', 'Galesville Ct', 'Gallant Ct', 'Galloway Ave', 'Galvaston Loop', 'Gannon Ave N', 'Gannon Ave S', 'Gansevoort Blvd', 'Garden Ct', 'Garden St', 'Gardenia Ln', 'Garfield Ave', 'Garibaldi Ave', 'Garretson Ave', 'Garretson Ln', 'Garrison Ave', 'Garson Ave', 'Garth Ct', 'Gary Ct', 'Gary Pl', 'Gary St', 'Gateway Dr', 'Gaton St', 'Gauldy Ave', 'Gaynor St', 'Geigerich Ave', 'Geldner Ave', 'General St', 'Genesee Ave', 'Genesee Ave', 'Genesee Ave', 'Genesee St', 'Gentile Ct', 'George St', 'Georges Ln', 'Georgia Ct', 'Gervil St', 'Getz Ave', 'Geyser Dr', 'Giacomo Ln', 'Gianna Ct', 'Gibson Ave', 'Giegerich Ave', 'Giffords Glen', 'Giffords Ln', 'Gigi St', 'Gil Ct', 'Gilbert Pl', 'Gilbert St', 'Giles Pl', 'Gillard Ave', 'Gilroy St', 'Gina Ct', 'Giordan Ct', 'Girard St', 'Gladwin Ave', 'Gladwin St', 'Glascoe Ave', 'Glen Ave', 'Glen St', 'Glendale Ave', 'Glenn Rd', 'Glenwood Ave', 'Glenwood Pl', 'Globe Ave', 'Gloria Ct', 'Glover St', 'Goethals Ave', 'Goethals Bridge', 'Goethals Rd N', 'Goff Ave', 'Gold Ave', 'Golfview Ct', 'Goller Pl', 'Goodall St', 'Goodell Ave', 'Goodwin Ave', 'Gordon Pl', 'Gordon St', 'Gorge Rd', 'Governor Rd', 'Gower St', 'Grace Ct', 'Grace Rd', 'Grafe St', 'Graham Ave', 'Graham Blvd', 'Grand Ave', 'Grandview Ave', 'Grandview Terrace', 'Granite Ave', 'Grant Pl', 'Grant St', 'Grantwood Ave', 'Grasmere Ave', 'Grasmere Ct', 'Grasmere Dr', 'Graves St', 'Gray St', 'Grayson St', 'Great Kills Rd', 'Greaves Ave', 'Greaves Ct', 'Greaves Ln', 'Greeley Ave', 'Green St', 'Green Valley Rd', 'Greencroft Ave', 'Greencroft Ln', 'Greenfield Ave', 'Greenfield Ct', 'Greenleaf Ave', 'Greenport St', 'Greentree Ln', 'Greenway Ave', 'Greenway Dr', 'Greenwood Ave', 'Gregg Pl', 'Gregory Ln', 'Greta Pl', 'Gridley Ave', 'Grille Ct', 'Grimsby St', 'Grissom Ave', 'Griswold Ct', 'Groton St', 'Grove Ave', 'Grove Pl', 'Grove St', 'Grymes Hill Rd', 'Guilford St', 'Guinevere Ln', 'Gulf Ave', 'Gunton Pl', 'Gurdon St', 'Gurley Ave', 'Guyon Ave', 'Gwenn Loop', 'Hafstrom St', 'Hagaman Pl', 'Hale St', 'Hales Ave', 'Hall Ave', 'Hallister St', 'Halpin Ave', 'Hamden Ave', 'Hamilton Ave', 'Hamilton Ave', 'Hamilton St', 'Hamlin Pl', 'Hammock Ln', 'Hampton Green', 'Hampton Pl', 'Hancock St', 'Hank Pl', 'Hannah St', 'Hanover Ave', 'Harbor Ln', 'Harbor Loop', 'Harbor Rd', 'Harbor View Ct', 'Harbor View Pl', 'Harbor View Pl E', 'Harborview Pl', 'Harborview Pl S', 'Harbour Ct', 'Hardin Ave', 'Hardy Pl', 'Hardy St', 'Hargold Ave', 'Harold Ave', 'Harold St', 'Harriet Ave', 'Harris Ave', 'Harris Ln', 'Harrison Ave', 'Harrison Pl', 'Harrison St', 'Hart Ave', 'Hart Blvd', 'Hart Loop', 'Hart Pl', 'Hartford Ave', 'Hartford St', 'Harvard Ave', 'Harvest Ave', 'Harvey Ave', 'Harvey St', 'Hasbrouck Rd', 'Hastings Ct', 'Hastings St', 'Hatfield Pl', 'Haughwout Ave', 'Haven Ave', 'Haven Esplanade', 'Havenwood Rd', 'Hawley Ave', 'Hawthorne Ave', 'Hay St', 'Haynes St', 'Haywood St', 'Hazen St', 'Headquarters Rd', 'Heaney Ave', 'Heather Ct', 'Heberton Ave', 'Hecker St', 'Heenan Ave', 'Heffernan St', 'Heinz Ave', 'Helena Rd', 'Helene Ct', 'Helios Pl', 'Hemlock Ct', 'Hemlock Ln', 'Hemlock St', 'Hempstead Ave', 'Henderson Ave', 'Hendricks Ave', 'Henning St', 'Henry Pl', 'Herbert St', 'Hereford St', 'Herkimer St', 'Herrick Ave', 'Hervey St', 'Hett Ave', 'Heusden St', 'Hewitt Ave', 'Hickory Ave', 'Hickory Cir', 'Hickory Ct', 'High St', 'Highland Ave', 'Highland Ln', 'Highland Rd', 'Highmount Rd', 'Highpoint Rd', 'Highview Ave', 'Hill St', 'Hillbrook Ct', 'Hillbrook Dr', 'Hillcrest Ave', 'Hillcrest Ct', 'Hillcrest St', 'Hillcrest Terrace', 'Hilldale Ct', 'Hillis St', 'Hillman Ave', 'Hillridge Ct', 'Hillside Ave', 'Hillside Terrace', 'Hilltop Pl', 'Hilltop Rd', 'Hilltop Terrace', 'Hillview Ln', 'Hillview Pl', 'Hillwood Ct', 'Hinton St', 'Hirsch Ln', 'Hitchcock Ave', 'Hoda Pl', 'Hodges Pl', 'Holbernt Ct', 'Holcomb Ave', 'Holden Blvd', 'Holdridge Ave', 'Holgate St', 'Holiday Dr', 'Holiday Way', 'Holland Ave', 'Holly Ave', 'Holly Pl', 'Holly St', 'Holsman Rd', 'Holton Ave', 'Home Ave', 'Home Pl', 'Homer St', 'Homestead Ave', 'Honey Ln', 'Hooker Pl', 'Hooper Ave', 'Hope Ave', 'Hope Ln', 'Hopkins Ave', 'Hopping Ave', 'Housman Ave', 'Houston Ln', 'Houston St', 'Howard Ave', 'Howard Cir', 'Howard Ct', 'Howe St', 'Howton Ave', 'Hoyt Ave', 'Hudson Pl', 'Hudson Rd', 'Hudson St', 'Huguenot Ave', 'Hull Ave', 'Humbert St', 'Hunt Ln', 'Hunter Ave', 'Hunter Pl', 'Hunter St', 'Hunton St', 'Hurlbert St', 'Huron Pl', 'Hurst St', 'Husson St', 'Hyatt St', 'Hygeia Pl', 'Hylan Blvd', 'Hylan Pl', 'Ibsen Ave', 'Ida Ct', 'Idaho Ave', 'Idlease Pl', 'Igros Ct', 'Ilion Pl', 'Ilyse Ct', 'Ilyssa Way', 'Impala Ct', 'Ina St', 'Indale Ave', 'Independence Ave', 'Industrial Loop', 'Industry Rd', 'Inez St', 'Ingram Ave', 'Innis St', 'Interstate 278', 'Inwood Rd', 'Iona St', 'Ionia Ave', 'Iowa Pl', 'Irene Ln', 'Iris Ct', 'Irma Pl', 'Iron Mine Dr', 'Ironwood St', 'Iroquois St', 'Irving Pl', 'Irvington St', 'Isabella Ave', 'Isernia Ave', 'Islington St', 'Ismay St', 'Isora Pl', 'Ithaca St', 'Ivan Ct', 'Ivy Ct', 'Jackson Ave', 'Jackson St', 'Jacob St', 'Jacques Ave', 'Jaffe St', 'Jake Ct', 'James Pl', 'Jamie Ct', 'Jamie Ln', 'Jansen Ct', 'Jansen St', 'Jardine Ave', 'Jarvis Ave', 'Jason Ct', 'Jasper St', 'Jay St', 'Jayne Ln', 'Jeanette Ave', 'Jeannette Ave', 'Jefferson Ave', 'Jefferson Blvd', 'Jefferson St', 'Jeffrey Pl', 'Jenna Ln', 'Jennifer Ct', 'Jennifer Ln', 'Jennifer Pl', 'Jerome Ave', 'Jerome Rd', 'Jersey St', 'Jessica Ct', 'Jessica Ln', 'Jewett Ave', 'Jillian Ct', 'Joan Pl', 'Joel Pl', 'Johanna Ln', 'John St', 'Johnathan Ct', 'Johnson Ave', 'Johnson Pl', 'Johnson St', 'Johnston Terrace', 'Jojo Ct', 'Joline Ave', 'Joline Ln', 'Jonathan Ct', 'Jones Pl', 'Jones St', 'Joseph Ave', 'Joseph Ct', 'Joseph Ln', 'Josephine St', 'Joshua Ct', 'Journeay Ave', 'Journeay St', 'Joyce Ln', 'Joyce St', 'Judith Ct', 'Jules Dr', 'Julie Ct', 'Julieann Ct', 'Jumel St', 'Juni Ct', 'Juniper Pl', 'Jupiter Ln', 'Justin Ave', 'Kaltenmeier Ln', 'Kalver Pl', 'Kansas Ave', 'Karen Ct', 'Katan Ave', 'Katan Loop', 'Kathleen Ct', 'Kathy Ct', 'Kathy Pl', 'Kay Pl', 'Keating Pl', 'Keating St', 'Keats St', 'Keegans Ln', 'Keeley St', 'Keiber Ct', 'Kell Ave', 'Kelly Blvd', 'Kelvin Ave', 'Kemball Ave', 'Kenilworth Ave', 'Kenmore St', 'Kenneth Pl', 'Kennington St', 'Kensico St', 'Kensington Ave', 'Kent St', 'Kenwood Ave', 'Keppel Ave', 'Kermit Ave', 'Kerry Ln', 'Keune Ct', 'Kimberly Ln', 'King James Ct', 'King St', 'Kingdom Ave', 'Kinghorn St', 'Kingsbridge Ave', 'Kingsland St', 'Kingsley Ave', 'Kingsley Pl', 'Kinsey Pl', 'Kirby Ct', 'Kirkland Ct', 'Kirshon Ave', 'Kissam Ave', 'Kissel Ave', 'Kiswick St', 'Klondike Ave', 'Knapp St', 'Knauth Pl', 'Knesel St', 'Knight Loop', 'Knollwood Ct', 'Knox Pl', 'Knox St', 'Koch Blvd', 'Korean War Veterans Pkwy', 'Kramer Ave', 'Kramer Pl', 'Kramer St', 'Kreischer St', 'Krissa Ct', 'Kristen Ct', 'Kruser St', 'Kunath Ave', 'Kyle Ct', 'Labau Ave', 'Laconia Ave', 'Ladd Ave', 'Lafayette Ave', 'Lafayette St', 'Laforge Ave', 'Laforge Pl', 'Lagrange Pl', 'Laguardia Ave', 'Laguna Ln', 'Lake Ave', 'Lakeland Rd', 'Lakeside Pl', 'Lakeview Terrace', 'Lakewood Rd', 'Lambert St', 'Lamberts Ln', 'Lamoka Ave', 'Lamont Ave', 'Lamped Ct', 'Lamped Loop', 'Lamport Blvd', 'Lander Ave', 'Landis Ave', 'Langere Pl', 'Lansing St', 'Larch Ct', 'Laredo Ave', 'Larkin St', 'Larrison Loop', 'Lasalle St', 'Latham Pl', 'Lathrop Ave', 'Latimer Ave', 'Latourette Ln', 'Latourette St', 'Laurel Ave', 'Laurie Ct', 'Lava St', 'Law Pl', 'Lawn Ave', 'Lawrence Ave', 'Layton Ave', 'Leason Pl', 'Ledyard Pl', 'Lee Ave', 'Leeds St', 'Leewood Loop', 'Legate Ave', 'Leggett Pl', 'Legion Pl', 'Leigh Ave', 'Lenevar Ave', 'Lenhart St', 'Lennon Ct', 'Lenore Ct', 'Lenzie St', 'Leo St', 'Leon St', 'Leona St', 'Leonard Ave', 'Leonard St', 'Leonello Ln', 'Lerer Ln', 'Leroy St', 'Leslie Ave', 'Lester St', 'Leverett Ave', 'Leverett Ct', 'Levinson Ln', 'Levit Ave', 'Lewiston St', 'Lexa Pl', 'Lexington Ave', 'Lexington Ln', 'Leyden Ave', 'Liberty Ave', 'Lighthouse Ave', 'Lightner Ave', 'Lilac Ct', 'Lillian Pl', 'Lillie Ln', 'Lily Pond Ave', 'Lincoln Ave', 'Lincoln Pl', 'Lincoln St', 'Linda Ave', 'Linda Ct', 'Linda Ln', 'Lindbergh Ave', 'Linden Ave', 'Linden Pl', 'Linden St', 'Lindenwood Pl', 'Lindenwood Rd', 'Link Rd', 'Linton Pl', 'Linwood Ave', 'Lion St', 'Lipsett Ave', 'Lisa Ln', 'Lisa Pl', 'Lisbon Pl', 'Lisk Ave', 'Liss St', 'Little Clove Rd', 'Little Clove Road Exd', 'Littlefield Ave', 'Livermore Ave', 'Livingston Ave', 'Livingston Ave', 'Livingston Ct', 'Llewellyn Pl', 'Lloyd Ct', 'Lockman Ave', 'Lockman Loop', 'Lockwood Pl', 'Locust Ave', 'Locust Ct', 'Locust Pl', 'Logan Ave', 'Lois Pl', 'Lombard Ct', 'London Rd', 'Long Pond Ln', 'Longdale St', 'Longfellow Ave', 'Longview Rd', 'Loop Rd', 'Loretto St', 'Loring Ave', 'Lorraine Ave', 'Lorraine Loop', 'Lortel Ave', 'Lott Ln', 'Louis St', 'Louise Ln', 'Louise St', 'Lovelace Ave', 'Lovell Ave', 'Lowell St', 'Lucille Ave', 'Lucy Loop', 'Ludlow St', 'Ludwig Ln', 'Ludwig St', 'Luigi Pl', 'Luke Ct', 'Lulu Ct', 'Luna Cir', 'Lundi Ct', 'Lundsten Ave', 'Luten Ave', 'Lyceum Ct', 'Lyle Ct', 'Lyman Ave', 'Lyman Pl', 'Lynbrook Ave', 'Lynbrook Ct', 'Lynch St', 'Lyndale Ave', 'Lyndale Ln', 'Lynhurst Ave', 'Lynn Ct', 'Lynn St', 'Lynnhaven Pl', 'Lyon Pl', 'Mac Gregor St', 'Mace St', 'Macfarland Ave', 'Macon Ave', 'Macormac Pl', 'Madan Ct', 'Madera St', 'Madigan Pl', 'Madison Ave', 'Madsen Ave', 'Magnolia Ave', 'Maguire Ave', 'Maguire Ct', 'Maiden Ln', 'Main St', 'Maine Ave', 'Majestic Ave', 'Major Ave', 'Malden Pl', 'Mallard Ln', 'Mallory Ave', 'Mallow St', 'Malone Ave', 'Maloney Dr', 'Malvine Ave', 'Manchester Dr', 'Mandy Ct', 'Manee Ave', 'Manhattan St', 'Manila Ave', 'Manila Pl', 'Manley St', 'Mann Ave', 'Manor Ct', 'Manor Rd', 'Mansion Ave', 'Manton Pl', 'Maple Ave', 'Maple Ct', 'Maple Pkwy', 'Maple Terrace', 'Mapleton Ave', 'Maplewood Ave', 'Maplewood Pl', 'Marble St', 'Marc St', 'Marcy Ave', 'Maretzek Ct', 'Margaret St', 'Margaretta Ct', 'Margo Loop', 'Maria Ln', 'Marianne St', 'Marie St', 'Marine Dr', 'Marine Way', 'Mariners Ln', 'Marion Ave', 'Marion St', 'Marisa Cir', 'Marisa Ct', 'Marjorie St', 'Mark St', 'Market St', 'Markham Dr', 'Markham Ln', 'Markham Pl', 'Markham Rd', 'Markham Walk', 'Marne Ave', 'Marscher Pl', 'Marsh Ave', 'Marshall Ave', 'Marshall Rd', 'Martha St', 'Martin Ave', 'Martineau St', 'Martling Ave', 'Marvin Rd', 'Marx St', 'Mary St', 'Maryland Ave', 'Maryland Ln', 'Maryland Pl', 'Mason Ave', 'Mason Blvd', 'Mason St', 'Massachusetts St', 'Mathews Ave', 'Matthew Pl', 'Maxi Ct', 'May Ave', 'May Pl', 'Mayberry Promenade', 'Maybury Ave', 'Maybury Ct', 'Mazza Ct', 'Mc Arthur Ave', 'Mc Baine Ave', 'Mc Cully Ave', 'Mc Divitt Ave', 'Mc Donald Ct', 'Mc Laughlin St', 'Mc Laughlin St', 'Mc Veigh Ave', 'Mcclean Ave', 'Mccormick Pl', 'Mccully Ave', 'Mcdonald St', 'Mckee Ave', 'Mckinley Ave', 'Meade Loop', 'Meadow Ave', 'Meadow Ct', 'Meadow Ln', 'Meadow Pl', 'Meagan Loop', 'Medford Rd', 'Medina St', 'Meeker St', 'Meisner Ave', 'Melba St', 'Melhorn Rd', 'Melissa St', 'Melrose Ave', 'Melrose Pl', 'Melville Rd', 'Melville St', 'Melvin Ave', 'Melyn Pl', 'Memo St', 'Memphis Ave', 'Mendelsohn St', 'Mercer Pl', 'Mercury Ln', 'Meredith Ave', 'Merivale Ln', 'Merkel Pl', 'Merle Pl', 'Merrick Ave', 'Merrill Ave', 'Merriman Ave', 'Merrymount St', 'Mersereau Ave', 'Metcalfe St', 'Metropolitan Ave', 'Meyer Ln', 'Michael Ct', 'Michael Loop', 'Michelle Ct', 'Michelle Ln', 'Mickardan Ct', 'Middle Loop Rd', 'Midland Ave', 'Midland Rd', 'Midway Pl', 'Milbank Rd', 'Milburn St', 'Mildred Ave', 'Miles Ave', 'Milford Ave', 'Milford Dr', 'Mill Ln', 'Mill Rd', 'Millennium Loop', 'Miller Field Rd', 'Miller St', 'Mills Ave', 'Millstone Ct', 'Milton Ave', 'Mimosa Ln', 'Minerva Ave', 'Minerva Pl', 'Minna St', 'Minthorne St', 'Minturn Ave', 'Mirlinda Ct', 'Mitchel Ln', 'Mitchell Ln', 'Mobile Ave', 'Moffett St', 'Mohn Pl', 'Mohr St', 'Monahan Ave', 'Monroe Ave', 'Monroe Pl', 'Monsey Pl', 'Mont Sec Ave', 'Montauk Pl', 'Montell St', 'Monterey Ave', 'Montgomery Ave', 'Montgomery Ave', 'Monticello Terrace', 'Montreal Ave', 'Montvale Pl', 'Moody Pl', 'Moore St', 'Morani St', 'Moreland St', 'Morgan Ln', 'Morley Ave', 'Morningstar Rd', 'Morris Pl', 'Morris St', 'Morrison Ave', 'Morrow St', 'Morse Ave', 'Morton St', 'Mosel Ave', 'Mosel Loop', 'Moselj', 'Mosely Ave', 'Mosley Ave', 'Motley Ave', 'Mott St', 'Mountainside Rd', 'Mountainview Ave', 'Mulberry Ave', 'Mulberry Cir', 'Muldoon Ave', 'Muller Ave', 'Mundy Ave', 'Murdock Pl', 'Muriel St', 'Murray Hulbert Ave', 'Murray Pl', 'Murray St', 'Myrna Ln', 'Myrtle Ave', 'N Bridge St', 'N Burgher Ave', 'N Carolina Ave', 'N Carolina Rd', 'N Edo Ct', 'N Gannon Ave', 'N Mada Ave', 'N Pine Terrace', 'N Railroad Ave', 'N Railroad Blvd', 'N Railroad St', 'N Randall Ave', 'N Rd', 'N Rhett Ave', 'N St Austins Pl', 'N Weed Rd', 'Nadal Pl', 'Nadine St', 'Nahant St', 'Nancy Ct', 'Nancy Ln', 'Naples St', 'Narrows Rd N', 'Narrows Rd S', 'Nash Ct', 'Nashville St', 'Naso Ct', 'Nassau Pl', 'Nassau Pl', 'Nassau St', 'Nassau St', 'Natalie Ct', 'Nathan Ct', 'Natick St', 'Naughton Ave', 'Nautilus St', 'Navesink Pl', 'Navigator Ct', 'Neal Dow Ave', 'Neckar Ave', 'Nedra Ln', 'Nedra Pl', 'Nehring Ave', 'Nelson Ave', 'Neptune Pl', 'Neptune St', 'Nesmythe Terrace', 'Netherland Ave', 'Neutral Ave', 'Nevada Ave', 'New Dorp Ln', 'New Dorp Plaza', 'New Folden Pl', 'New Ln', 'New Mexico Ct', 'New St', 'New York Ave', 'New York Pl', 'Newark Ave', 'Newberry Ave', 'Newkirk Ave', 'Newton St', 'Newvale Ave', 'Niagara St', 'Nicholas Ave', 'Nicholas St', 'Nichols Ct', 'Nick Laporte Pl', 'Nicole Loop', 'Nicolosi Dr', 'Nicolosi Loop', 'Nielsen Ave', 'Nightingale St', 'Niles Pl', 'Nina Ave', 'Nippon Ave', 'Nixon Ave', 'Noah Ct', 'Noble Pl', 'Noel St', 'Nolan Ave', 'Nome Ave', 'Norden St', 'Norma Pl', 'Normalee Rd', 'Norman Pl', 'Norman St', 'North Ave', 'North Dr', 'North St', 'Northentry Rd', 'Northern Blvd', 'Northfield Ave', 'Northfield Ct', 'Northport Ln', 'Northview Ct', 'Norwalk Ave', 'Norway Ave', 'Norwich St', 'Norwood Ave', 'Norwood Ct', 'Nostrand Ave', 'Notre Dame Ave', 'Notus Ave', 'Nugent Ave', 'Nugent Ct', 'Nugent St', 'Nunley Ct', 'Nunzie Ct', 'Nutly Pl', 'Nutwood Ct', "O'gorman Ave", 'Oak Ave', 'Oak Ct', 'Oak Ln', 'Oak St', 'Oakdale Ave', 'Oakdale St', 'Oakland Ave', 'Oakland Terrace', 'Oakley Pl', 'Oakville St', 'Oakwood Ave', 'Oban St', 'Oberlin St', 'Occident Ave', 'Ocean Ave', 'Ocean Ct', 'Ocean Driveway', 'Ocean Rd', 'Ocean Terrace', 'Oceanic Ave', 'Oceanside Ave', 'Oceanview Ave', 'Oceanview Ln', 'Oceanview Pl', 'Oconnor Ave', 'Odell Pl', 'Oder Ave', 'Odin St', 'Ogden St', 'Ohio Pl', 'Old Amboy Rd', 'Old Farmers Ln', 'Old Mill Rd', 'Old Town Rd', 'Oldfield St', 'Olga Pl', 'Olive St', 'Oliver Pl', 'Olivia Ct', 'Olympia Blvd', 'Omaha St', 'Onda Ct', 'Oneida Ave', 'Oneida Ave', 'Oneida Ave', 'Ontario Ave', 'Ontario Ave', 'Opal Ln', 'Opp Ct', 'Opus Ct', 'Orange Ave', 'Orange Ave', 'Orange Ave', 'Orange Ave', 'Orbit Ln', 'Orchard Ave', 'Orchard Ln', 'Orchard St', 'Ordell Ave', 'Oregon Rd', 'Orinoco Pl', 'Orlando St', 'Ormond Pl', 'Ormsby Ave', 'Osage Ln', 'Osborn Ave', 'Osborne St', 'Osgood Ave', 'Ostrich Ct', 'Oswald Pl', 'Oswego St', 'Oswego St', 'Otis Ave', 'Otsego Ave', 'Ottavio Promenade', 'Outerbridge Ave', 'Outerbridge Crossing', 'Ovas Ct', 'Overlook Ave', 'Overlook Dr', 'Overlook Terrace', 'Ovis Pl', 'Oxford Pl', 'Oxholm Ave', 'Pacific Ave', 'Page Ave', 'Palace Ct', 'Palisade St', 'Palma Dr', 'Palmer Ave', 'Palmieri Ln', 'Pamela Ln', 'Papasso Way', 'Paradise Pl', 'Paris Ct', 'Parish Ave', 'Park Ave', 'Park Ct', 'Park Dr', 'Park Dr N', 'Park Hill Ave', 'Park Hill Cir', 'Park Hill Ct', 'Park Hill Ln', 'Park Ln', 'Park Pl', 'Park Rd', 'Park St', 'Park Terrace', 'Parker St', 'Parkinson Ave', 'Parkview Loop', 'Parkview Pl', 'Parkwood Ave', 'Parsons Pl', 'Patten St', 'Patterson Ave', 'Patty Ct', 'Paulding Ave', 'Pauw St', 'Pavillion Hill Terrace', 'Paxton St', 'Peacock Loop', 'Peare Pl', 'Pearl St', 'Pearsall St', 'Pearson St', 'Pebble Ln', 'Peck Ct', 'Peel Pl', 'Peggy Ln', 'Pelican Cir', 'Pelton Ave', 'Pelton Pl', 'Pemberton Ave', 'Pembrook Loop', 'Penbroke Ave', 'Pendale St', 'Pendleton Pl', 'Penn Ave', 'Penn St', 'Pennsylvania Ave', 'Penton St', 'Percival Pl', 'Perine Ave', 'Perkiomen Ave', 'Perry Ave', 'Pershing St', 'Perth Amboy Pl', 'Peru St', 'Peter Ave', 'Peter Ct', 'Peter St', 'Petersons Ln', 'Petrus Ave', 'Petunia Ct', 'Pheasant Ln', 'Phelps Pl', 'Philip Ave', 'Phyllis Ct', 'Piave Ave', 'Pickersgill Ave', 'Piedmont Ave', 'Pierce St', 'Pierpont Pl', 'Pierre Pl', 'Pike St', 'Pilcher St', 'Pine Pl', 'Pine St', 'Pine Terrace', 'Pinewood Ave', 'Pitney Ave', 'Pitt Ave', 'Pittsville Ave', 'Plank Rd', 'Platinum Ave', 'Platt St', 'Plattsburg St', 'Pleasant Ct', 'Pleasant Pl', 'Pleasant Plains Ave', 'Pleasant St', 'Pleasant Valley Ave', 'Plymouth Rd', 'Poe Ct', 'Poe St', 'Poets Cir', 'Poi Ct', 'Poillon Ave', 'Point St', 'Poland Pl', 'Polk Walk', 'Pommer Ave', 'Pompey Ave', 'Pond St', 'Pond Way', 'Pontiac St', 'Poplar Ave', 'Poplar Ln', 'Port Ln', 'Port Richmond Ave', 'Portage Ave', 'Portland Pl', 'Portsmouth Ave', 'Posen St', 'Post Ave', 'Post Ln', 'Potter Ave', 'Pouch Terrace', 'Poultney St', 'Powell Ln', 'Powell St', 'Prague Ct', 'Prall Ave', 'Pratt Ct', 'Prescott Ave', 'President St', 'Presley St', 'Preston Ave', 'Prices Ln', 'Primrose Pl', 'Prince Ln', 'Prince St', 'Princess Ln', 'Princess St', 'Princeton Ave', 'Princeton Ln', 'Princewood Ave', 'Prol Pl', 'Promenade Ave', 'Property St', 'Prospect Ave', 'Prospect Pl', 'Prospect St', 'Providence St', 'Pulaski Ave', 'Pumone Ct', 'Purcell St', 'Purdue Ct', 'Purdue St', 'Purdy Ave', 'Purdy Pl', 'Purroy Pl', 'Putnam Pl', 'Putnam St', 'Putter Ct', 'Putters Ct', 'Quail Ln', 'Queen St', 'Queensdale St', 'Quimby Ave', 'Quincy Ave', 'Quinlan Ave', 'Quinn St', 'Quintard St', 'Rachel Ct', 'Radcliff Rd', 'Radford St', 'Radigan Ave', 'Rae Ave', 'Ragazzi Ln', 'Railroad Ave', 'Railroad St', 'Raily Ct', 'Rainbow Ave', 'Ralph Ave', 'Ralph Pl', 'Ramapo Ave', 'Ramble Rd', 'Ramblewood Ave', 'Ramona Ave', 'Ramsey Ln', 'Ramsey Pl', 'Randall Ave', 'Range Rd', 'Rankin St', 'Raritan Ave', 'Rathbun Ave', 'Ravenhurst Ave', 'Ravenna St', 'Rawson Pl', 'Ray St', 'Rayfield Ct', 'Raymond Ave', 'Raymond Pl', 'Reading Ave', 'Rear Amherst Ave', 'Rector St', 'Red Cedar Ln', 'Redgrave Ave', 'Redmond Ave', 'Redwood Ave', 'Redwood Loop', 'Regal Walk', 'Regan Ave', 'Regent Cir', 'Regina Ln', 'Regis Dr', 'Reid Ave', 'Reiss Ln', 'Remsen St', 'Rene Dr', 'Renee Pl', 'Renfrew Pl', 'Reno Ave', 'Rensselaer Ave', 'Rensselaer Ave', 'Renwick Ave', 'Reon Ave', 'Retford Ave', 'Retner St', 'Revere Ln', 'Revere St', 'Reynaud St', 'Reynolds Ct', 'Reynolds St', 'Rhett Ave', 'Rhine Ave', 'Rice Ave', 'Rich Valley Rd', 'Richard Ave', 'Richard Ln', 'Riche Ave', 'Richmond Av - Armand St', 'Richmond Ave', 'Richmond Ct', 'Richmond Hill Rd', 'Richmond Pkwy', 'Richmond Pl', 'Richmond Rd', 'Richmond Terrace', 'Richmond Terrace', 'Richmond Valley Rd', 'Ridge Ave', 'Ridge Ct', 'Ridge Loop', 'Ridgecrest Ave', 'Ridgefield Ave', 'Ridgeway Ave', 'Ridgewood Ave', 'Ridgewood Pl', 'Riedel Ave', 'Riegelmann St', 'Riga St', 'Rigamar Ct', 'Rigby Ave', 'Rigby St', 'Rigimar Ct', 'Riley Pl', 'Ring Rd', 'River Rd', 'Rivington Ave', 'Roanoke St', 'Robert Ln', 'Roberts Dr', 'Robin Ct', 'Robin Rd', 'Robinson Ave', 'Rocco Ct', 'Rochelle Pl', 'Rochelle St', 'Rockaway St', 'Rockland Ave', 'Rockne St', 'Rockport Ave', 'Rockville Ave', 'Rockwell Ave', 'Rodeo Ln', 'Roderick Ave', 'Rodman St', 'Roe St', 'Roff St', 'Rogers Pl', 'Rokeby Pl', 'Rolling Hill Green', 'Roma Ave', 'Roman Ave', 'Roman Ct', 'Rome Ave', 'Romer Rd', 'Romier Rd', 'Ronald Ave', 'Roosevelt Ave', 'Roosevelt Ct', 'Roosevelt St', 'Rose Ave', 'Rose Ct', 'Rose Ln', 'Rosebank Pl', 'Rosecliff Rd', 'Rosedale Ave', 'Rosewood Pl', 'Ross Ave', 'Ross Ln', 'Rossville Ave', 'Roswell Ave', 'Row Pl', 'Rowan Ave', 'Roxbury St', 'Royal Oak Rd', 'Royal St', 'Rubenstein St', 'Rudyard St', 'Rugby Ave', 'Ruggles St', 'Rumba Pl', 'Rumpler Ct', 'Rumson Rd', 'Rupert Ave', 'Russek Dr', 'Russell St', 'Rustic Pl', 'Ruth Pl', 'Ruth St', 'Rutherford Ct', 'Ruxton Ave', 'Ryan Pl', 'Rye Ave', 'S 5th St', 'S Beach Ave', 'S Bridge St', 'S Drum St', 'S Edo Ct', 'S Gannon Ave', 'S Goff Ave', 'S Greenleaf Ave', 'S I Mall Dr', 'S Mann Ave', 'S Railroad Ave', 'S Railroad St', 'S Rd', 'S Service Rd', 'S St Austins Pl', 'S Weed Rd', 'Sable Ave', 'Sable Loop', 'Sabrina Ln', 'Saccheri Ct', 'Sage Ct', 'Sagona Ct', 'Sala Ct', 'Salamander Ct', 'Sally Ct', 'Salvatore Papasso Way', 'Salvatore T Papasso Way', 'Salzburg Ct', 'Samantha Ln', 'Sampson Ave', 'Samuel Pl', 'Sand Ct', 'Sand Ln', 'Sandalwood Dr', 'Sandborn St', 'Sanders St', 'Sandgap St', 'Sandra Ln', 'Sands St', 'Sandy Ln', 'Sandywood Ln', 'Sanford Pl', 'Sanford St', 'Sanilac St', 'Santa Monica Ln', 'Santo Ct', 'Sapphire Ct', 'Sarcona Ct', 'Satterlee St', 'Saturn Ln', 'Saunder St', 'Savin Ct', 'Savo Ln', 'Savo Loop', 'Savoy St', 'Sawyer Ave', 'Saxon Ave', 'Saybrook St', 'Scarboro Ave', 'Scarsdale St', 'Scenic Ln', 'Scheffelin Ave', 'Schindler Ct', 'Schley Ave', 'Schmidts Ln', 'Schoharie St', 'School Rd', 'School St', 'Schooner Loop', 'Schubert St', 'Schuyler St', 'Scott Ave', 'Scranton Ave', 'Scranton St', 'Scribner Ave', 'Scudder Ave', 'Sea Breeze Ln', 'Sea Gate Rd', 'Seabreeze Ln', 'Seacrest Ave', 'Seacrest Ln', 'Seafoam St', 'Seagate Ct', 'Search Ln', 'Seaside Ln', 'Seaver Ave', 'Seaview Ave', 'Seeley Ln', 'Seely Ln', 'Seguine Ave', 'Seguine Loop', 'Seguine Pl', 'Seidman Ave', 'Seldin Ave', 'Selkirk St', 'Selvin Loop', 'Seneca Ave', 'Seneca Loop', 'Seneca St', 'Serrell Ave', 'Seth Ct', 'Seth Loop', 'Seward Pl', 'Seymour Ave', 'Shadow Ln', 'Shadyside Ave', 'Shafter Ave', 'Shaina Ct', 'Shale Ln', 'Shale St', 'Sharon Ave', 'Sharon Ln', 'Sharpe Ave', 'Sharrett Pl', 'Sharrott Ave', 'Sharrotts Ln', 'Sharrotts Rd', 'Shaughnessy Ln', 'Shaw Pl', 'Shawnee St', 'Sheffield St', 'Sheldon Ave', 'Shell Rd', 'Shelley Ave', 'Shelterview Dr', 'Shenandoah Ave', 'Shepard Ave', 'Sheraden Ave', 'Sheridan Ave', 'Sheridan Ct', 'Sheridan Pl', 'Sherill Ave', 'Sherman Ave', 'Sherrill Ave', 'Sherwood Ave', 'Sherwood Pl', 'Shiel Ave', 'Shields Pl', 'Shift Pl', 'Shiloh St', 'Shirley Ave', 'Shirra Ave', 'Shore Acres Rd', 'Shore Ave', 'Shore Rd', 'Shore St', 'Short Pl', 'Shotwell Ave', 'Si Mall Dr', 'Sideview Ave', 'Sierra Ct', 'Signal Hill Rd', 'Signs Rd', 'Silver Beech Rd', 'Silver Ct', 'Silver Lake Park Rd', 'Silver Lake Rd', 'Simmons Ln', 'Simmons Loop', 'Simon Ct', 'Simonson Ave', 'Simonson Pl', 'Simpson St', 'Sinclair Ave', 'Singleton St', 'Sioux St', 'Sky Ln', 'Skyline Dr', 'Slaight St', 'Slater Blvd', 'Slayton Ave', 'Sleepy Hollow Rd', 'Sleight Ave', 'Sleight St', 'Sloane Ave', 'Slosson Ave', 'Slosson Terrace', 'Smith Ave', 'Smith Ct', 'Smith Pl', 'Smith St', 'Smith Terrace', 'Smyrna Ave', 'Sneden Ave', 'Snug Harbor Rd', 'Sobel Ct', 'Sommer Ave', 'Sommers Ln', 'Sonia Ct', 'Sophia Ln', 'Soren St', 'South Ave', 'South St', 'Spanish Camp', 'Spanish Colony', 'Spanish Ln', 'Spark Pl', 'Sparkill Ave', 'Sparta Pl', 'Spartan Ave', 'Speedwell Ave', 'Spencer St', 'Sperry Pl', 'Sperry St', 'Spirit Ln', 'Sprague Ave', 'Sprague Ct', 'Spratt Ave', 'Sprauge', 'Spring St', 'Springfield Ave', 'Springhill Ave', 'Spruce Ct', 'Spruce Ln', 'Spruce St', 'St Adalbert Pl S', 'St Albans Pl', 'St Andrews Pl', 'St Andrews Rd', 'St Anthony Pl', 'St Austins Pl', 'St Edward Ln', 'St George Dr', 'St George Rd', 'St James Pl', 'St John Ave', 'St Johns Ave', 'St Josephs Ave', 'St Julian Pl', 'St Lukes Ave', 'St Marks Pl', 'St Marys Ave', 'St Patricks Pl', 'St Pauls Ave', 'St Peters Pl', 'St Stephens Pl', 'Stacey Ln', 'Stack Dr', 'Stafford Ave', 'Stage Ln', 'Stanley Ave', 'Stanley Cir', 'Stanwich St', 'Star Ct', 'Starbuck St', 'Starlight Rd', 'Starr Ave', 'State Highway 440', 'State Route 440', 'State St', 'Staten Is Blvd', 'Staten Is Mall Dr', 'Staten Island Blvd', 'Station Ave', 'Stebbins Ave', 'Stecher St', 'Steele Ave', 'Steers St', 'Steinway Ave', 'Stephen Loop', 'Stepney St', 'Sterling Ave', 'Stern Ct', 'Steuben St', 'Stevenson Pl', 'Stewart Ave', 'Stieg Ave', 'Stobe Ave', 'Stone Crest Ct', 'Stone Ln', 'Stone St', 'Stonegate Dr', 'Stoneham St', 'Storer Ave', 'Stratford Ave', 'Stratford Ct', 'Strauss St', 'Strawberry Ln', 'Stroud Ave', 'Studio Ln', 'Sturges St', 'Stuyvesant Ave', 'Stuyvesant Pl', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Suffolk Ave', 'Summer St', 'Summerfield Pl', 'Summit Ave', 'Summit Pl', 'Summit Rd', 'Summit St', 'Sumner Ave', 'Sumner Pl', 'Sunfield Ave', 'Sunnymeade Village', 'Sunnyside Terrace', 'Sunrise Terrace', 'Sunset Ave', 'Sunset Hill Dr', 'Sunset Ln', 'Surf Ave', 'Surfside Plaza', 'Susan Ct', 'Susana Ln', 'Susanna Ln', 'Sussex Ave', 'Sussex Green', 'Sutton Pl', 'Swaim Ave', 'Swan St', 'Sweetbrook Rd', 'Sweetwater Ave', 'Swinnerton St', 'Sycamore St', 'Sydney Pl', 'Sylva Ln', 'Sylvan Ct', 'Sylvan Pl', 'Sylvaton Terrace', 'Sylvia St', 'Tabb Pl', 'Tacoma St', 'Taft Ave', 'Taft Ct', 'Talarico Ct', 'Talbot Pl', 'Tallman St', 'Tanglewood Dr', 'Tappen Ct', 'Targee St', 'Tarla Ct', 'Tarlee Pl', 'Tarlton St', 'Tarring St', 'Tarrytown Ave', 'Tate St', 'Tatro St', 'Taunton St', 'Taxter Pl', 'Taylor Ct', 'Taylor St', 'Teakwood Ct', 'Ted Pl', 'Teleport Dr', 'Temple Ct', 'Tenafly Pl', 'Tennyson Dr', 'Teri Ct', 'Terrace Ave', 'Terrace Ct', 'Tessa Ct', 'Thames Ave', 'Thayer Pl', 'The Blvd', 'The Oval', 'The Plaza', 'Theater Ln', 'Thelma Ct', 'Theresa Pl', 'Thistle Ct', 'Thollen St', 'Thomas Pl', 'Thomas St', 'Thompson Pl', 'Thompson St', 'Thornycroft Ave', 'Thurston St', 'Tiber Pl', 'Tides Ln', 'Tiffany Ct', 'Tiger Ct', 'Tilden St', 'Tiller Ct', 'Tillman St', 'Tilson Pl', 'Timber Ridge Dr', 'Timothy Ct', 'Tioga St', 'Tioga St', 'Titus Ave', 'Toddy Ave', 'Todt Hill Ct', 'Todt Hill Rd', 'Token St', 'Tom Ct', 'Tompkins Ave', 'Tompkins Ave', 'Tompkins Cir', 'Tompkins Ct', 'Tompkins Pl', 'Tompkins St', 'Tone Ln', 'Tonking Rd', 'Tony Ct', 'Topping St', 'Topside Ln', 'Totten St', 'Tottenville Pl', 'Towers Ln', 'Townley Ave', 'Townsend Ave', 'Tracy Ave', 'Trantor Pl', 'Travis Ave', 'Treadwell Ave', 'Treetz Pl', 'Tremont Ave', 'Trent St', 'Trenton Ct', 'Tricia Way', 'Trina Ln', 'Trinity Pl', 'Trossach Rd', 'Trout Pl', 'Troy St', 'Truman St', 'Trumbull Pl', 'Tryon Ave', 'Tuckahoe Ave', 'Tudor St', 'Tulip Cir', 'Turf Ct', 'Turf Rd', 'Turner St', 'Tuscany Ct', 'Tuttle St', 'Twin Oak Dr', 'Twombly Ave', 'Tyler Ave', 'Tynan St', 'Tyndale St', 'Tyrellan Ave', 'Tyrrell St', 'Tysen Ct', 'Tysen St', 'Tysens Ln', 'Uncas Ave', 'Union Ave', 'Union Ct', 'Union Pl', 'University Pl', 'Upton St', 'Urbana St', 'Usak Ct', "Usak's Ct", 'Uss Arizona Ln', 'Uss Connecticut Ct', 'Uss Florida Ct', 'Uss Iowa Cir', 'Uss Missouri Ln', 'Uss New Mexico Ct', 'Uss North Carolina Rd', 'Uss Tennessee Rd', 'Utah St', 'Utica St', 'Utopia Ct', 'Utter Ave', 'Uxbridge St', 'Vail Ave', 'Vail Pl', 'Valdemar Ave', 'Valencia Ave', 'Valley Rd', 'Valleyview Pl', 'Van Allen Ave', 'Van Brunt St', 'Van Buren St', 'Van Cortlandt Ave', 'Van Duzer St', 'Van Duzer St Exn', 'Van Name Ave', 'Van Pelt Ave', 'Van Riper St', 'Van St', 'Van Tuyl St', 'Van Wyck Ave', 'Vanderbilt Ave', 'Vanessa Ln', 'Vassar St', 'Vaughan St', 'Vedder Ave', 'Veith Pl', 'Veltman Ave', 'Venice Ave', 'Venus Ln', 'Venus Pl', 'Vera St', 'Vermont Ave', 'Vermont Ct', 'Vernon Ave', 'Verrazzano Bridge Plaza', 'Vespa Ave', 'Veterans Rd E', 'Veterans Rd W', 'Victoria Rd', 'Victory Blvd', 'Vienna Ct', 'Villa Ave', 'Village Ct', 'Village Ln', 'Villanova St', 'Vincent Ave', 'Vine St', 'Vineland Ave', 'Virginia Ave', 'Virginia Ave', 'Virginia Pl', 'Vista Ave', 'Vista Pl', 'Vogel Ave', 'Vogel Ln', 'Vogel Loop', 'Vogel Pl', 'Von Braun Ave', 'Vreeland St', 'Vulcan St', 'W Buchanan St', 'W Castor Pl', 'W Caswell Ave', 'W Cedarview Ave', 'W Fingerboard Rd', 'W Main St', 'W Raleigh Ave', 'W Service Rd', 'W Shore Expy', 'W Shore Exwy E State Route', 'W Shore Exwy State Route E', 'W Shore Exwy State Route West', 'Wade St', 'Wadsworth Ave', 'Wadsworth Rd', 'Wadsworth Terrace', 'Wagner St', 'Wahler Pl', 'Waimer Pl', 'Wainwright Ave', 'Wakefield Ave', 'Wakefield Rd', 'Walbrooke Ave', 'Walch Pl', 'Walcott Ave', 'Walden Ave', 'Waldo Pl', 'Waldron Ave', 'Wales Pl', 'Walker Ct', 'Walker Dr', 'Walker Pl', 'Walker St', 'Wall St', 'Wallace Ave', 'Walloon St', 'Walnut Ave', 'Walnut Pl', 'Walnut St', 'Walters Ave', 'Wandel Ave', 'Ward Ave', 'Wards Point Ave', 'Wardwell Ave', 'Waring Ave', 'Warren St', 'Warwick Ave', 'Washington Ave', 'Washington Pl', 'Watchogue Rd', 'Water St', 'Waterbury Ave', 'Waterford Ct', 'Waters Ave', 'Waterside Pkwy', 'Waterside St', 'Waterview Ct', 'Watkins Ave', 'Watson Ave', 'Wave St', 'Wavecrest St', 'Waverly Pl', 'Wayne Ct', 'Wayne Pl', 'Wayne St', 'Wayne St', 'Weaver St', 'Webster Ave', 'Weed Ave', 'Weiner St', 'Weir Ave', 'Weir Ln', 'Wellbrook Ave', 'Welles Ct', 'Wellington Ct', 'Wemple St', 'Wendy Dr', 'Wenlock St', 'Wentworth Ave', 'Weser Ave', 'West Shore Expressway Service Rd E', 'West Shore Expressway Service Rd W', 'West Shore Exwy W State Route', 'West St', 'West Terrace', 'Westbrook Ave', 'Westbury Ave', 'Westcott Blvd', 'Westentry Rd', 'Western Ave', 'Westervelt Ave', 'Westfield Ave', 'Westminster Ct', 'Westport Ln', 'Westport St', 'Westwood Ave', 'Wetmore Rd', 'Whalley Ave', 'Wheeler Ave', 'Wheeling Ave', 'Whitaker Pl', 'White Ct', 'White Oak Ln', 'White Pl', 'White Plains Ave', 'White St', 'Whitehall St', 'Whitewood Ave', 'Whitlock Ave', 'Whitman Ave', 'Whitney Ave', 'Whitwell Pl', 'Wiederer Pl', 'Wieland Ave', 'Wilbur Pl', 'Wilbur St', 'Wilcox St', 'Wild Ave', 'Wilder Ave', 'Wildwood Ln', 'Wiley Pl', 'Willard Ave', 'Willard Pl', 'William Ave', 'William St', 'Willis Ave', 'Willow Ave', 'Willow Ln', 'Willow Pond Rd', 'Willow Rd E', 'Willow Rd W', 'Willow Road W Exd', 'Willow Wood Ln', 'Willowbrook Ct', 'Willowbrook Parkway', 'Willowbrook Rd', 'Wills Pl', 'Wilson Ave', 'Wilson St', 'Wilson Terrace', 'Wilsonview Pl', 'Wilton Ct', 'Wiman Ave', 'Wiman Pl', 'Winans St', 'Winant Ave', 'Winant Ln', 'Winant Pl', 'Winant St', 'Winchester Ave', 'Windemere Ave', 'Windermere Rd', 'Windham Loop', 'Winding Woods Loop', 'Windmill Ct', 'Windom Ave', 'Windsor Ct', 'Windsor Rd', 'Windy Hollow Way', 'Winegar Ln', 'Winfield Ave', 'Winfield St', 'Wingham St', 'Winham Ave', 'Winslow Pl', 'Winston St', 'Winter Ave', 'Winthrop Pl', 'Wirt Ave', 'Wirt Ln', 'Witteman Pl', 'Woehrle Ave', 'Wolcott Ave', 'Wolkoff Ln', 'Wolverine St', 'Wood Ave', 'Wood Ct', 'Wood Ln', 'Woodbine Ave', 'Woodbridge Pl', 'Woodcliff Ave', 'Woodcrest Rd', 'Woodcutters Ln', 'Wooddale Ave', 'Woodhaven Ave', 'Woodhull Ave', 'Woodland Ave', 'Woodlawn Ave', 'Woodrose Ln', 'Woodrow Rd', 'Woodruff Ln', 'Woods Of Arden Rd', 'Woodside Ave', 'Woodstock Ave', 'Woodvale Ave', 'Woodvale Ave Exd', 'Woodvale Loop', 'Woodward Ave', 'Woolley Ave', 'Wrenn St', 'Wright Ave', 'Wright St', 'Wycliff Ln', 'Wygant Pl', 'Wyona Ave', 'Xenia St', 'Yacht Club Cove', 'Yacht Club Cove St', 'Yafa Ct', 'Yaffa Ct', 'Yale St', 'Yeomalt Ave', 'Yeshiva Ln', 'Yetman Ave', 'Yona Ave', 'York Ave', 'York Terrace', 'Young St', 'Yucca Dr', 'Yukon Ave', 'Zachary Ct', 'Zebra Pl', 'Zeck Ct', 'Zeluff Ave', 'Zephyr Ave', 'Zoe St', 'Zwicky Ave']

In [ ]:
def create_random_numbers(min_value, max_value, break_value, small_numbers, large_numbers):
    '''
    Generates a list of random numbers within specified ranges.
    Args:
        min_value (int): The minimum value.
        max_value (int): The maximum value.
        break_value (int): The boundary value that separates small and large numbers, must be > min_value and < max_value
        small_numbers (int): The number of small numbers.
        large_numbers (int): The number of large numbers.
    '''
    if break_value <= min_value or break_value >= max_value:
        raise ValueError("break_value must be greater than min_value and smaller than max_value")

    small_numbers_set = [random.randrange(min_value, break_value) for _ in range(small_numbers)]
    large_numbers_set = [random.randrange(break_value, max_value) for _ in range(large_numbers)]

    random_numbers = small_numbers_set + large_numbers_set
    return random_numbers


In [ ]:
random_numbers = create_random_numbers(1, 5898, 100, 900, 50)

In [ ]:
def create_random_length_list():
    
    list_of_strings = random.sample(streets, random.choice(random_numbers) + 1)
    list_as_string = "[" + ", ".join(list_of_strings) + "]"

    return list_as_string

In [ ]:
# Convert a function to UDF
random_length_list_udf = udf(create_random_length_list, StringType())

In [ ]:
df_with_street_list = df_updated.withColumn("street_list", random_length_list_udf())

### 5. Save as parquet with Spark partitions

In [ ]:
partition_columns = ["year", "month"]

In [ ]:
df_with_street_list.write.partitionBy(partition_columns).mode("overwrite").parquet(f'{raw_base_path}spark-partitions-yellow')

### 6. Save as parquet with manual partitions

In [ ]:
# read from the spark partitions DF to ensure the data frames are the same
df_manual = spark.read.parquet(f'{raw_base_path}spark-partitions-yellow')

In [ ]:
df_manual.cache().count()

Out[23]: 45779591

In [ ]:
# optionally - modify and change the save mode to append
def partition_manually(year, month):
    month = str(month).zfill(2)
    file_name = f'{raw_base_path}manual-prefix-yellow/{year}/{month}'
    df = df_manual.filter((df_manual['year'] == year) & (df_manual['month'] == month))
    df.write.parquet(file_name)


In [ ]:
def run_partition_manually():
    years = [2021, 2020]
    months = range(1, 13)

    for year in years:
        for month in months:
            partition_manually(year, month)

In [ ]:
run_partition_manually()